A lo largo del trabajo se podrá seguir un en los markdown un índice con la secuencia de pasos que fuimos transitando. Distinguiendo a través del tamaño si se trata de pasos o subpasos.

Además, en color rojo se llama la atención respecto de los tres modelados llevados adelante.

Por último, en color azul dejamos algunas preguntas que nos fueron surgieron y que nos sería de provecho poder aproximar un respuesta.

# 1) Importo librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, KFold

from sklearn.model_selection import GridSearchCV

# 2) Importo fuente de datos como data frame y exploro brevemente

In [2]:
ubicacion = 'https://github.com/Ftroyer/repositorio_tp_phyton_uba/raw/main/insumo_nuevo_tp_phyton_uba.xlsx'

data = pd.read_excel(ubicacion)

print(data.shape)

print(data.columns)

(6037, 170)
Index(['nro_formulario', 'Establecimiento', 'Carrera', 'genero',
       'fecha_nacimiento', 'localidad_nacimiento', 'dia_inscripcion',
       'nivel_conocimiento_materias_plan_estudio',
       'nivel_conocimiento_duracion_carrera',
       'nivel_conocimiento_funciones_graduado',
       ...
       'actividad_medios_museos', 'actividad_medios_cine',
       'actividad_medios_podcast_radio', 'actividad_medios_club',
       'actividad_medios_organizacion_estudiantil',
       'expectativa_lugar_de_trabajo', 'expectativa_rol', 'obra_social_alu',
       'materias_inscripcion', 'materias_aprobadas'],
      dtype='object', length=170)


In [3]:
data.head(5)

,nro_formulario,Establecimiento,Carrera,genero,fecha_nacimiento,localidad_nacimiento,dia_inscripcion,nivel_conocimiento_materias_plan_estudio,nivel_conocimiento_duracion_carrera,nivel_conocimiento_funciones_graduado,...,actividad_medios_museos,actividad_medios_cine,actividad_medios_podcast_radio,actividad_medios_club,actividad_medios_organizacion_estudiantil,expectativa_lugar_de_trabajo,expectativa_rol,obra_social_alu,materias_inscripcion,materias_aprobadas
0,1,fsoc_unsam,Ciencias de la comunicación,Femenino,2006-02-21,Mataderos,5,3. Algún conocimiento,4. Mucho conocimiento,3. Algún conocimiento,...,No,No,No,No,No,1. Estatal,Docente frente a alumnos,Está incluido/ a en la cobertura de salud de s...,1,0
1,2,fsoc_unsam,Ciencias de la comunicación,Femenino,1998-03-11,Oruro,47,3. Algún conocimiento,4. Mucho conocimiento,4. Mucho conocimiento,...,No,No,No,No,Si,3. Ambas,Docente frente a alumnos,Como afiliado voluntario (a obra social o prep...,4,0
2,3,fsoc_unsam,Trabajo social,Femenino,1999-12-19,BARRIO SAN JUAN (BERAZATEGUI-PDO. BERAZATEGUI),39,2. Poco conocimiento,4. Mucho conocimiento,3. Algún conocimiento,...,No,No,No,No,No,NaN,NC,Carece de cobertura de salud,1,0
3,4,fsoc_unsam,Ciencias de la comunicación,Femenino,2006-01-10,Villa Lugano,8,3. Algún conocimiento,3. Algún conocimiento,4. Mucho conocimiento,...,No,No,No,No,No,3. Ambas,Docente frente a alumnos,Está incluido/ a en la cobertura de salud de s...,4,0
4,5,fsoc_unsam,Trabajo social,Femenino,1995-12-04,Villa Lugano,53,3. Algún conocimiento,3. Algún conocimiento,3. Algún conocimiento,...,No,No,No,No,No,1. Estatal,Docente frente a alumnos,Como afiliado voluntario (a obra social o prep...,5,0


# 3) Chequeo existencia de nulos y chequeo existencia de duplicados

3.a) Chequeo existencia de registros nulos

Y recategorizo algunas variables que asumen ciertos valores equivalentes a nulos:

El data frame es el resultante, entre otras cosas, de un cuestionario autodministrado. Como consecuencia de ello
tiene distintos valores con los que se identifican respuestas faltantes por distintos motivos:

"NR" --> No responde, cuando debiera haberlo hecho;

"9999" --> No responde, cuando debiera haberlo hecho;

"NC" --> No corresponde, es decir que como consecuencia de un pase de un formulario esa persona no debía responder esa pregunta;

"NA"

(además, hay un conjunto de valores tales como "..." o "-" que nosotros no identificamos pero que, en realidad, son respuestas no válidas)

In [4]:
nulos = data.isnull().sum()

print(nulos.sort_values(ascending=False))

valores_nulos = ["NC", "NR", "9999", "NA", 9999]

data = data.replace(valores_nulos, np.nan)

nulos = data.isnull().sum()

print(nulos.sort_values(ascending=False))

alu_idioma_otro          5818
nivel_idioma_chino       2398
nivel_idioma_aleman      2363
nivel_idioma_italiano    2338
nivel_idioma_frances     2315
                         ... 
cant_fami_cargo             0
padre_vive                  0
ult_est_cur_padre           0
sit_laboral_padre           0
materias_aprobadas          0
Length: 170, dtype: int64


C:\Users\facundo.troyer\AppData\Local\Temp\ipykernel_8840\1953461770.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace(valores_nulos, np.nan)


alu_otestsup_razon_abandono1    6037
alu_otestsup_egre2              6013
alu_otestsup_razon_abandono2    5994
alu_otestsup_carr2              5957
alu_otestsup_niv2               5949
                                ... 
alu_cos_est_ap_fam                 0
tipo_vivienda                      0
desafio_otra                       0
desafio_incer_fin                  0
materias_aprobadas                 0
Length: 170, dtype: int64


3.b) Chequeo existencia de ¿filas? duplicadas

In [5]:
data.duplicated().sum()

0

<span style="color:red">Vamos a modelar 3 veces:</span>

<span style="color:red">La primera con el data frame modificado sólo en lo estrictamente necesario para poder modelar.</span>

<span style="color:red">La segunda sometiendo a la base a feature selection y feature engineering en función de nuestro conocimiento de las variables.</span>

<span style="color:red">La tercera repitiendo el feature selection pero ahora en base al feature importance y optimizando hiperparametros con grid search.</span>

<span style="color:red">Primer modelado</span>

En este modelado el procesamiento va a ser elemental. La idea principal es poder ver las métricas de evaluación que alcanzamos con el data frame sin procesamiento para, luego, ver si mejoramos como consecuencia de nuestro procesamiento.

# 4) Preliminar EDA y Feature Selection y Feature Engineering

4.a) Renombro y recategorizo variables 

Acá, como dijimos, vamos a hacer lo mínimo y necesario. Este paso tomará su verdadera importanca en el segundo modelado.

No obstante, estos pocos procesamientos los conservaremos para todo el proyecto porque son elementales para poder avanzar en el modelado.

No lo haremos sobre el data frame original, sino sobre data_1.

In [6]:
data_1 = data.copy()

------

<span style="color:blue">El data frame con el que trabajamos tiene una gran proorción de respuestas categóricas y, si bien optamos por una forma de trabajo nos queda la duda de si fue el adecuado:

<span style="color:blue">Por un lado, hay ciertas variables categóricas que tienen muchas opciones distintas de respuesta. En un momento decidimos dejarlas, pero luego encontramos que al transformarlas a dummies se traducían en muchisimas variables. En otro momento buscamos pensamos reemplazar sus valores categóricos por números, pero no nos convenció. Finalmenteo decidimos eliminar aquellas variables con muchos tipos de respuesta distinto. ¿El procedimiento por el que optamos fue el adecuado o hubiese sido mejor hacer otra cosa?

<span style="color:blue">Por otro lado, muchas de las variables de este data frame son respuestas a un formulario. Por el propio transcurrir del formulario hay preguntas que respondían sólo algunas de las personas. Por ejemplo, las preguntas relativas a la situación laboral son muy parecidas a las de la EPH, entonces cuando la persona declara tener trabajo pasa a un conjunto de preguntas, mientras que cuando dice no tener trabajo pasa a otro conjunto de preguntas. No se trata de que la persona dejó la pregunta en blanco, sino que para cada perfil laboral corresponde un circuito distinto de preguntas. Con ello, todos los que no tienen trabajo no responden, por caso, si son obreros o patrones. Luego los patrones no responden la cantidad de horas trabajadas.

<span style="color:blue">Esto nos exigió pensar qué hacer con los valores nulos. Al comienzo imputamos valores sin más y nos encontrabamos que variables con gran proporción de valores nulos terminaba por tener puras respuestas imputadas. Ahí decidimos eliminar aquellas variables que tenían muchos valores nulos y, sólo después, imputar valores. Nos queda la duda si podríamos haber recurrido a una alternativa mejor. ¿Por caso, hubiese sido adecuado juntar variables? Volviendo al ejemplo de la situación laboral una sola variable podría contener para distintos registros: No trabaja, pero busca / Patron / Trabaja 40 horas por semana.</span>

Acá vamos a ir en búsqueda de aquellas variables categóricas que tienen un número muy elevado de respuestas únicas y las eliminaremos.

Por un lado, porque de esa forma identificaremos fácilmente las columnas que son el resultado de preguntas con respuesta libre del formulario que alimenta parte de este data frame. Por caso, el nombre del colegio secundario, donde cada quien respondía escribiendo dicho nombre.

Por otro lado, porque como luego transformaremos las variables categóricas a dummies, tener variables con muchas respuestas únicas resulta problemático.

In [7]:
variables_categoricas_iniciales = data_1.select_dtypes(include=['object', 'category']).columns
variables_categoricas_iniciales

Index(['Establecimiento', 'Carrera', 'genero', 'localidad_nacimiento',
       'nivel_conocimiento_materias_plan_estudio',
       'nivel_conocimiento_duracion_carrera',
       'nivel_conocimiento_funciones_graduado',
       'nivel_conocimiento_mercado_laboral',
       'nivel_conocimiento_salario_docente',
       'nivel_conocimiento_estatuto_docente',
       ...
       'actividad_medios_actividades_sociales_politicas_barriales',
       'actividad_medios_teatro', 'actividad_medios_museos',
       'actividad_medios_cine', 'actividad_medios_podcast_radio',
       'actividad_medios_club', 'actividad_medios_organizacion_estudiantil',
       'expectativa_lugar_de_trabajo', 'expectativa_rol', 'obra_social_alu'],
      dtype='object', length=152)

In [8]:
unicos_por_categoria = {col: data_1[col].nunique() for col in variables_categoricas_iniciales}

unicos_df = pd.DataFrame(list(unicos_por_categoria.items()), columns=['Variable', 'Unicos'])

unicos_df = unicos_df.sort_values(by='Unicos', ascending=False).reset_index(drop=True)

print(unicos_df.head(20))

                               Variable  Unicos
0                    alu_otestsup_carr4     997
1                  localidad_nacimiento     630
2                    alu_otestsup_carr1     398
3                     titulo_secundario     365
4                    alu_otestsup_carr2      77
5                     alu_cos_est_espec      72
6                       alu_idioma_otro      64
7                    alu_otestsup_egre4      37
8                    alu_otestsup_egre1      29
9                    alu_otestsup_egre2      14
10                    ult_est_cur_padre      11
11                    ult_est_cur_madre      11
12                              Carrera      10
13         alu_otestsup_razon_abandono4       9
14         alu_otestsup_razon_abandono2       9
15                   alu_otestsup_area1       8
16                   alu_otestsup_area4       8
17                   alu_otestsup_area2       8
18             franja_horaria_trab_alum       7
19  estas_finalizando_secu_terminalidad 

In [9]:
data_1 = data_1.drop([# categóricas
                      'alu_otestsup_carr4',
                      'localidad_nacimiento',
                      'alu_otestsup_carr1',
                      'titulo_secundario',
                      'alu_otestsup_carr2',
                      'alu_cos_est_espec',
                      'alu_idioma_otro',
                      'alu_otestsup_egre4',
                      'alu_otestsup_egre1',
                      'alu_otestsup_egre2',
                      # esta por no considerarla relevante
                      'dia_inscripcion', 
                      # esta porque es un id
                      'nro_formulario',
                      # esta por considerar que se solapa con la edad
                      'anio_egreso_sec',
                      # estas porque nos dio la sensación de que no era pertinente a la investigación dejarlas, pero no estamos seguros
                      'Establecimiento',
                      'Carrera'], axis=1)


<span style="color:blue">Sumamos otra duda: ¿es apropiado borrar Establecimiento y Carrera? Nosotros decidimos sacarlas porque como estamos buscando las cualidades de los ingresantes que desertan nos pareció que estaba de más. Nos queda la duda si no hubiese sido mejor dejarlas para, si así fuera, que el modelo pudiera sumarlo como cualidades Por caso, que evaluase cierta carrera como característica significativa a la hora de modelar la deserción temprana de los ingresantes.</span>

-----

In [10]:
# proceso fecha_nacimiento para dar con r_edad porque tengo que desestimar fecha_nacimiento antes de modelar por el tipo de dato

# Queremos calcular la edad al momento del ingreso.
# Por eso restamos "2023-11-10" a la fecha de nacimiento.
# Esa es la fecha en que se respondió la encuesta.

data_1['fecha_nacimiento'] = pd.to_datetime(data_1['fecha_nacimiento'])

data_1['r_edad'] = ((pd.Timestamp('2023-11-10') - data_1['fecha_nacimiento']).dt.days/365).astype(int) 

data_1[['fecha_nacimiento', 'r_edad']].sample(n=10)

,fecha_nacimiento,r_edad
56,2023-07-08,0
5296,2002-03-12,21
3177,1992-10-20,31
388,2005-05-05,18
1311,2004-05-09,19
5103,2005-08-31,18
2635,1973-10-14,50
6003,1992-02-15,31
917,1968-02-08,55
5326,2002-03-09,21


In [11]:
data_1 = data_1.drop(columns=['fecha_nacimiento'])

print(data_1.shape)

(6037, 155)


-----

In [12]:
# acá creo la que será nuestra target

materias = ['materias_inscripcion', 'materias_aprobadas']

data_1['r_desercion_temprana'] = data_1.apply(
    lambda x: 'Deserto' if (x[materias] == 0).any() 
    else 'Sigue',
    axis=1)

print(data_1.shape)

data_1[materias + ['r_desercion_temprana']].sample(n=10)

(6037, 156)


,materias_inscripcion,materias_aprobadas,r_desercion_temprana
3277,2,0,Deserto
6031,2,1,Sigue
959,2,1,Sigue
708,4,4,Sigue
2332,4,3,Sigue
5555,3,2,Sigue
807,1,0,Deserto
5219,0,0,Deserto
4467,3,3,Sigue
3862,4,0,Deserto


In [13]:
data_1 = data_1.drop(columns=materias)

print(data_1.shape)

(6037, 154)


4.b) Identifico aun dentro de mi data frame la variable que en un futuro va a ser mi VD y reemplazo sus valores por números

In [14]:
data_1['r_desercion_temprana'].unique()

array(['Deserto', 'Sigue'], dtype=object)

In [15]:
data_1['r_desercion_temprana'] = data_1['r_desercion_temprana'].replace({'Deserto': 0,
                                                                         'Sigue': 1})

C:\Users\facundo.troyer\AppData\Local\Temp\ipykernel_8840\1400905226.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_1['r_desercion_temprana'] = data_1['r_desercion_temprana'].replace({'Deserto': 0,


In [16]:
data_1['r_desercion_temprana'].unique()

array([0, 1], dtype=int64)

4.c) Sustituyo nulos

*Hasta acá los cambios que sirven para todos los modelados.

*Para el segundo modelado tengo que volver hasta este paso.

In [17]:
# identifico nulos

nulos = data_1.isnull().sum()

print(nulos.sort_values(ascending=False))

alu_otestsup_razon_abandono1    6037
alu_otestsup_razon_abandono2    5994
alu_otestsup_niv2               5949
alu_otestsup_area2              5943
alu_otestsup_estado2            5941
                                ... 
alu_cos_est_plsoc                  0
alu_cos_est_ap_fam                 0
tipo_vivienda                      0
desafio_otra                       0
r_desercion_temprana               0
Length: 154, dtype: int64


In [18]:
# Además, reviso si no tengo alguna columna con todos valores nulos
# De existir la dropeo

variables_nulas = data_1.columns[data_1.isnull().all()]

print(variables_nulas)

print(data_1.shape)

data_1 = data_1.drop(columns=variables_nulas)

print(data_1.shape)

Index(['alu_otestsup_razon_abandono1'], dtype='object')
(6037, 154)
(6037, 153)


In [19]:
# ya sabemos que nuestra variable target no tiene nulos, así que no se le va a imputar nada

data_1_imputado = data_1.copy()

variables_categoricas_im = data_1.select_dtypes(include=['object', 'category']).columns

variables_numericas_im = data_1.select_dtypes(include=['int64', 'float64']).columns

imputador_categorico = SimpleImputer(strategy='most_frequent')  

imputador_numerico = SimpleImputer(strategy='mean')

data_1_imputado[variables_categoricas_im] = imputador_categorico.fit_transform(data_1[variables_categoricas_im])

data_1_imputado[variables_numericas_im] = imputador_numerico.fit_transform(data_1[variables_numericas_im])

print(data_1.shape)

print(data_1_imputado.shape)

nulos = data_1.isnull().sum()

print(nulos.sort_values(ascending=False))

(6037, 153)
(6037, 153)
alu_otestsup_razon_abandono2    5994
alu_otestsup_niv2               5949
alu_otestsup_area2              5943
alu_otestsup_estado2            5941
nivel_idioma_otro               5847
                                ... 
alu_cos_est_plsoc                  0
alu_cos_est_ap_fam                 0
tipo_vivienda                      0
desafio_otra                       0
r_desercion_temprana               0
Length: 153, dtype: int64


In [20]:
nulos = data_1_imputado.isnull().sum()

print(nulos.sort_values(ascending=False))

genero                       0
hace_dep_gim_priv            0
disp_actuales_tel            0
disp_actuales_ereader        0
disp_actuales_auriculares    0
                            ..
alu_trab_hace                0
alu_trab_ocup                0
hora_sem_trab_alum           0
vinculacion_docencia         0
r_desercion_temprana         0
Length: 153, dtype: int64


In [21]:
print(data_1.alu_trab_hace)

print(data_1_imputado.alu_trab_hace)

0                                  NaN
1       Obrero o empleado (asalariado)
2       Obrero o empleado (asalariado)
3                                  NaN
4       Obrero o empleado (asalariado)
                     ...              
6032                               NaN
6033                               NaN
6034    Obrero o empleado (asalariado)
6035    Obrero o empleado (asalariado)
6036                               NaN
Name: alu_trab_hace, Length: 6037, dtype: object
0       Obrero o empleado (asalariado)
1       Obrero o empleado (asalariado)
2       Obrero o empleado (asalariado)
3       Obrero o empleado (asalariado)
4       Obrero o empleado (asalariado)
                     ...              
6032    Obrero o empleado (asalariado)
6033    Obrero o empleado (asalariado)
6034    Obrero o empleado (asalariado)
6035    Obrero o empleado (asalariado)
6036    Obrero o empleado (asalariado)
Name: alu_trab_hace, Length: 6037, dtype: object


In [22]:
print(data_1.cuantos_medios_transportes_insume)

print(data_1_imputado.cuantos_medios_transportes_insume)

0       2.0
1       2.0
2       NaN
3       1.0
4       1.0
       ... 
6032    1.0
6033    1.0
6034    1.0
6035    2.0
6036    NaN
Name: cuantos_medios_transportes_insume, Length: 6037, dtype: float64
0       2.000000
1       2.000000
2       1.755698
3       1.000000
4       1.000000
          ...   
6032    1.000000
6033    1.000000
6034    1.000000
6035    2.000000
6036    1.755698
Name: cuantos_medios_transportes_insume, Length: 6037, dtype: float64


4.d) Elimino duplicados

In [23]:
# esto no vamos a dejar para el segundo modelo, cuando ya hayamos recurrido al procesamiento de variables

# 5) Identifico aun dentro de mi data frame las variables que en un futuro van a ser mis VIs y:

In [24]:
variables_independientes = data_1_imputado.columns.drop(['r_desercion_temprana'])
variables_independientes

Index(['genero', 'nivel_conocimiento_materias_plan_estudio',
       'nivel_conocimiento_duracion_carrera',
       'nivel_conocimiento_funciones_graduado',
       'nivel_conocimiento_mercado_laboral',
       'nivel_conocimiento_salario_docente',
       'nivel_conocimiento_estatuto_docente', 'familiar_ejerce_docencia',
       'mot_inst_publica', 'mot_inst_difusion',
       ...
       'actividad_medios_teatro', 'actividad_medios_museos',
       'actividad_medios_cine', 'actividad_medios_podcast_radio',
       'actividad_medios_club', 'actividad_medios_organizacion_estudiantil',
       'expectativa_lugar_de_trabajo', 'expectativa_rol', 'obra_social_alu',
       'r_edad'],
      dtype='object', length=152)

5.a) Distingo aun dentro de mi data frame VIs categóricas y VIs numéricas

In [25]:
# a traves de ".columns" me quedo sólo con el nombre de las columnas, de lo contrario me devuelve una salida tipo data frame

variables_categoricas = data_1_imputado[variables_independientes].select_dtypes(include=['object', 'category']).columns
variables_categoricas

Index(['genero', 'nivel_conocimiento_materias_plan_estudio',
       'nivel_conocimiento_duracion_carrera',
       'nivel_conocimiento_funciones_graduado',
       'nivel_conocimiento_mercado_laboral',
       'nivel_conocimiento_salario_docente',
       'nivel_conocimiento_estatuto_docente', 'familiar_ejerce_docencia',
       'mot_inst_publica', 'mot_inst_difusion',
       ...
       'actividad_medios_actividades_sociales_politicas_barriales',
       'actividad_medios_teatro', 'actividad_medios_museos',
       'actividad_medios_cine', 'actividad_medios_podcast_radio',
       'actividad_medios_club', 'actividad_medios_organizacion_estudiantil',
       'expectativa_lugar_de_trabajo', 'expectativa_rol', 'obra_social_alu'],
      dtype='object', length=140)

In [26]:
variables_numericas = data_1_imputado[variables_independientes].select_dtypes(include=[np.int32, np.int64, np.float64]).columns
print(variables_numericas)

print("Acá toma todas las 'mot_prop' como numéricas pero no hacemos nada porque es el primer modelado. Para el segundo modelado lo resolvemos")

Index(['mot_prop_sugerencia_nucleo_familiar',
       'mot_prop_interes_ejercer_docencia', 'mot_prop_gusto_trabajo_niños',
       'mot_prop_opcion_laboral_estable', 'mot_prop_remuneracion_docente',
       'mot_prop_interes_area_disciplinar',
       'mot_prop_asigno_valor_conocimiento',
       'mot_prop_prestigio_social_profesion',
       'mot_prop_permite_trabajar_act_personales',
       'mot_prop_no_sabia_que_estudiar', 'cuantos_medios_transportes_insume',
       'r_edad'],
      dtype='object')
Acá toma todas las 'mot_prop' como numéricas pero no hacemos nada porque es el primer modelado. Para el segundo modelado lo resolvemos


5.b) Transformo aun dentro de mi data frame VIs categóricas a dummies

In [27]:
# dummies es la forma más tradicional de transformar variables categóricas a numéricas, por medio
# de crear una columna nueva por cada valor de la variable categórica.
# trabajo   condicion_trabajo
#   si           monotriburo
#   si           monotributo
#   si           relacion de dependencia
#   no              -

#  trabajo_si   trabajo_no   condicion_trabajo_monotributo     condicion_trabajo_relacion_dependencia
#     True         False              True                              False
#     True         False              True                              False
#     True         False              False                             True
#     False        False              False                             False

print("Entre data y data_1 tiene que incrementarse la cantidad de columnas")
print(data.shape)

data_1_imputado = pd.get_dummies(data_1_imputado, columns=variables_categoricas, drop_first=True)

print(data_1_imputado.shape)

Entre data y data_1 tiene que incrementarse la cantidad de columnas
(6037, 170)
(6037, 311)


5.c) Transformo aun dentro de mi data frame VIs numéricas a VIs numéricas escaladas

In [28]:
# ponderar sirve para que el modelo no se vea afectado por variables que, por su propia naturaleza, tienen valores mas altos (puesto 
# que podria ponderar mas esas variables)
# y sirve para no verse afectado por la distorción/ el desvio de las variables

# REVISAR ESTAS 2 AFIRMACIONES:
# parece que estandatiza entre -3 y 3
# Esto asegura que cada característica tenga una media de 0 y una desviación estándar de 1

scaler = StandardScaler()
data_1_imputado[variables_numericas] = scaler.fit_transform(data_1_imputado[variables_numericas])

print("Seguimos ante un conjunto de variables que a su interior tienen series")
print(type(data_1_imputado[variables_numericas]))
print(type(data_1_imputado[variables_numericas[0]]))
print("Vemos el cambio de valores dentro de las variales")
print(data[variables_numericas[0]])
print(data_1_imputado[variables_numericas[0]])
print("Vemos el cambio de promedios")
print(np.mean (data[variables_numericas[0]]))
print(np.mean (data_1_imputado[variables_numericas[0]]))

Seguimos ante un conjunto de variables que a su interior tienen series
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
Vemos el cambio de valores dentro de las variales
0        6.0
1        4.0
2        8.0
3        5.0
4        2.0
        ... 
6032     8.0
6033     9.0
6034     1.0
6035    10.0
6036     NaN
Name: mot_prop_sugerencia_nucleo_familiar, Length: 6037, dtype: float64
0       8.330994e-02
1      -5.734593e-01
2       7.400792e-01
3      -2.450747e-01
4      -1.230229e+00
            ...     
6032    7.400792e-01
6033    1.068464e+00
6034   -1.558613e+00
6035    1.396848e+00
6036   -2.916641e-16
Name: mot_prop_sugerencia_nucleo_familiar, Length: 6037, dtype: float64
Vemos el cambio de promedios
5.74630376344086
-2.2009523204760433e-16


# 6) Salgo de mi data frame y creo mis objetos nuevos y e X

In [29]:
# "axis=1" 
# la función de "axis=1" es dar cuenta que hacemos referencia a una columna.
# "axis=0", mientras tanto, da cuenta de que hablamos de una fila.
#
# en "y = data_1['Target']" no hace falta el "axis=1" porque esa enunciación es específica para seleccionar columnas
# en "X = data_1.drop('Target', axis=1)" se trabaja sobre un data frame y el método "drop" puede operar quitando filas o columnas,
# de allí que sea necesario especificar que nuestro deseo es quitar una columna mediante "axis=1" 

y = data_1_imputado['r_desercion_temprana']
X = data_1_imputado.drop('r_desercion_temprana', axis=1)

print("El tipo de data de y que almacena una lista con los valores de una sola variable es distinto del tipo de dato de X que almacena distintas variables")
print(type(y))
print(type(X))
print(y)
print(X.columns)

# tanto la division entre y e X, como la futura division entre train y test vale para modelos supervisados, no así para no supervisados

El tipo de data de y que almacena una lista con los valores de una sola variable es distinto del tipo de dato de X que almacena distintas variables
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6032    1.0
6033    1.0
6034    1.0
6035    0.0
6036    0.0
Name: r_desercion_temprana, Length: 6037, dtype: float64
Index(['mot_prop_sugerencia_nucleo_familiar',
       'mot_prop_interes_ejercer_docencia', 'mot_prop_gusto_trabajo_niños',
       'mot_prop_opcion_laboral_estable', 'mot_prop_remuneracion_docente',
       'mot_prop_interes_area_disciplinar',
       'mot_prop_asigno_valor_conocimiento',
       'mot_prop_prestigio_social_profesion',
       'mot_prop_permite_trabajar_act_personales',
       'mot_prop_no_sabia_que_estudiar',
       ...
       'expectativa_lugar_de_trabajo_3. Ambas',
       'expectativa_lugar_de_trabajo_4. Me imagino trabajando fuera de la educación formal',
       'expec

# 7) Hago el split train-test con mis objetos y e X

In [30]:
# con "stratify=y" le pedimos que conserve entre train y test la proporcion de registros con los distintos valores de y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91, stratify=y)

print("Vemos que los train tienen más filas que los test, pero tienen la misma cantidad de columnas")
print(y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(X_test.shape)
print("También vemos que y e X conservan su tipo de dato")
print(type(y_train))
print(type(y_test))
print(type(X_train))
print(type(X_test))

Vemos que los train tienen más filas que los test, pero tienen la misma cantidad de columnas
(4829,)
(1208,)
(4829, 310)
(1208, 310)
También vemos que y e X conservan su tipo de dato
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [31]:
# confirmo que sigo sin valores nulos

nulos_train = X_train.isnull().sum()
nulos_test = X_test.isnull().sum()

print("Valores nulos en X_train:")
print(nulos_train.sort_values(ascending=False))

print("Valores nulos en X_test:")
print(nulos_test.sort_values(ascending=False))

Valores nulos en X_train:
mot_prop_sugerencia_nucleo_familiar                                                0
alu_otestsup_razon_abandono2_Dificultades de aprendizaje                           0
alu_otestsup_razon_abandono2_Por trabajo                                           0
alu_otestsup_razon_abandono2_Por tareas de cuidado de menores o adultos a cargo    0
alu_otestsup_razon_abandono2_Por problemas de salud                                0
                                                                                  ..
padre_vive_Si                                                                      0
padre_vive_No                                                                      0
cant_fami_cargo_3. Más de dos                                                      0
cant_fami_cargo_2. Dos                                                             0
obra_social_alu_Por su propio trabajo                                              0
Length: 310, dtype: int64
Valores nulos

# 8) Hago el modelado inicial

<span style="color:blue">Al correr algunas líneas de código de este trabajo sale alguna advertencia, pero igual se alcanza el resultado. Sucede, por ejemplo, en la próxima ejecución de la regresión logística. ¿Está bien seguir adelante dado que el script se ejecuta igual o corresponde hacer lo necesario para que deje de salir dicha advertencia?


<span style="color:blue">Justo la ejecución de la regresión logística se resuelve en el segundo y en el tercero modelado y fue por eso que no hicimos nada, pero igual nos queda la duda.</span>

8.a) Creo una función que contiene el diseño general de: el modelo, su entrenamiento, su predicción y su evaluación, para luego ir ejecutandolo con cada uno de los modelos que necesito

In [32]:
# Trabajando por fuera de una función:
#   1) Defino el modelo:
#   model = LogisticRegression(max_iter=10000)
#   2) Entreno el modelo:
#   model.fit(X_train, y_train)
#   3) Predigo:
#   y_pred = model.predict(X_test)
#   4) Evaluo:
#   accuracy = accuracy_score(y_test, y_pred)
#   5) Veo el resultado de la evaluación:
#   accuracy
# 
# Mientras tanto, cuando modelamos por medio de crear una funcion dejamos la "1) Defino el modelo"
# para cuando llamamos a la función

def evaluate_model(model, X_train, X_test, y_train, y_test):
      
    # el if que viene a continuacion se hace exclusivamente por xgboost, que por no venir en sikit learn
    # exige una forma de balanceo de las variables de y distinto al del resto de los modelos.
    # mientras que los demas se balancean cuando designamos los hiperparametros a la hora de definir el modelo,
    # el xgboost debe ser balanceado en este momento:
    
    if isinstance(model, XGBClassifier):
        # Calcular los pesos de las clases
        class_weights = {i: sum(y_train != i) / sum(y_train == i) for i in np.unique(y_train)}
        # Crear una lista de sample weights
        sample_weights = np.array([class_weights[y] for y in y_train])
    else:
        sample_weights = None

     # 2) Entreno del modelo:
    model.fit(X_train, y_train) 
    
     # 3) Predigo:
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
     # 4) Evaluo - comparando nuestras predicciones (que se entrenaro con X_train e y_train y se predijeron con X_test) con y_test -:
    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted') # average='weighted' se utiliza cuando la y tiene más de 2 valores: deserto, esta cursando y egreso
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted') 
    # creo una tabla provisoria con los valores predichos y los valores de y_test
    data_prueba_pred = pd.DataFrame({
      'y_test': y_test,
      'y_pred': y_pred,
      'y_pred_proba': y_pred_proba,
      'residuo_al_cuadrado' : (y_test - y_pred) ** 2
    })
    # guardo un objeto con el tipo de validación cruzada "KFold" y luego lo ejecuto y almaceno en el objeto "cross_val_scores"
    kf = KFold(n_splits=5, # --> 5 particiones
           shuffle=True, # --> pedimos que las particiones se hagan por medio de aleatoriedad
           random_state=97)
    cross_val_scores = cross_val_score(model, 
                                   X, 
                                   y, 
                                   cv=kf, # --> hacer la validación cruzada a través del objeto en que almacenamos el KFold: "kf"
                                   scoring='accuracy') # --> designamos accuracy como la métrica con la que hacemos la validacion
    
     # 5) Veo el resultado de la evaluación:
    print("La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:")
    print(conf_matrix)
    print("El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:")
    print(accuracy)
    print("El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:")
    print(precision)
    print("El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:")
    print(recall)
    print("El F1 muestra la media armónica de la precisión y el recall:")
    print(f1)
    print("Visualización tabular de los resultados:")
    print(data_prueba_pred)
    print("Error cuadrático medio:")
    print(data_prueba_pred['residuo_al_cuadrado'].mean())
    print("Raiz del error cuadrático medio:")
    print((data_prueba_pred['residuo_al_cuadrado'].mean())**0.5)
    print("Valores de validación cruzada:")
    print(cross_val_scores)
    print("Promedio de valores de validación cruzada:")
    print(cross_val_scores.mean())

    return model, y_pred, accuracy, cross_val_scores.mean()  # el contenido del return es lo que se almacenará 
                         # como resultado de nuestra función cuando la ejecutemos 
                         # se almacenará en los objetos que ponemos antes de llamar a la función
                         # por caso, "dt_model, dt_pred = evaluate_model(...)"
                         
                         # la particularidad en esta función es que uno de esos resultados es a la vez uno de los inputs, mientras
                         # que el otro resultado no es un input:
                         # inputs --> model, X_train, X_test, y_train, y_test
                         # resultados --> model, y_pred
                         # por eso al ejecutar la función veremos:
                         # "dt_model, dt_pred = evaluate_model(dt_model, X_train, X_test, y_train, y_test)"

                         # si no nos hiciera falta que el modelo y la prediccion de y quedaran almacenados en objetos que podamos llamar 
                         # más adelante, la función podría no tener el "return" y, en cuyo caso, luego ejecutarla sin poner nada por delante:
                         # "evaluate_model(dt_model, X_train, X_test, y_train, y_test)"

8.b) Defino los modelos y ejecuto la función que acabo de crear para cada uno de los modelos definidos

Árbol de decisión

In [33]:
results = {}

In [34]:
# 1) Defino el modelo POR FUERA DE LA FUNCION:

dt_model = DecisionTreeClassifier(random_state=42)

dt_model, dt_pred, dt_accuracy, dt_cross_val_mean = evaluate_model(dt_model, X_train, X_test, y_train, y_test)

results['Arbol de decision'] = (dt_accuracy, dt_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[225 257]
 [324 402]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5190397350993378
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5301423843530751
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5190397350993378
El F1 muestra la media armónica de la precisión y el recall:
0.5230341100403938
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
4248     1.0     0.0           0.0                  1.0
3750     1.0     0.0           0.0                  1.0
292      1.0     1.0           1.0                  0.0
2895     1.0     0.0           0.0                  1.0
668      0.0     1.0           1.0                  1.

Random Forest

In [35]:
rf_model = RandomForestClassifier(random_state=42)    # con "class_weight = 'balance'" se están balanceado los valores de y
                                                                               # si tengo 95% que deserto y 5% que siguio, un modelo que dice que todos
                                                                               # dejaron andaria bastante bien. 
                                                                               # para evitar eso este hiperparametro asigna más peso o, lo que se lo mismo,
                                                                               # pondera cuando la prediccion la pega en el valor de y que es poco común.
                                                                               # (minimiza la funcion de pérdida cuando se predice correctamente esos casos)
f_model, rf_pred, rf_accuracy, rf_cross_val_mean = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

results['Random Forest'] = (rf_accuracy, rf_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 40 442]
 [ 58 668]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5860927152317881
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5245387433656119
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5860927152317881
El F1 muestra la media armónica de la precisión y el recall:
0.492359553245063
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
4248     1.0     1.0         0.650                  0.0
3750     1.0     1.0         0.645                  0.0
292      1.0     1.0         0.650                  0.0
2895     1.0     1.0         0.640                  0.0
668      0.0     1.0         0.660                  1.0

XG Boost

In [36]:
xgb_model = XGBClassifier(random_state=42) 

xgb_model, xgb_pred, xgb_accuracy, xgb_cross_val_mean = evaluate_model(xgb_model, X_train, X_test, y_train, y_test)

results['XG Boost'] = (xgb_accuracy, xgb_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[137 345]
 [191 535]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5562913907284768
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5320349095461153
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5562913907284768
El F1 muestra la media armónica de la precisión y el recall:
0.5353853838718818
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
4248     1.0       1      0.938328                  0.0
3750     1.0       0      0.227224                  1.0
292      1.0       1      0.710126                  0.0
2895     1.0       1      0.612862                  0.0
668      0.0       1      0.534545                  1.

Regresión Logística

In [37]:
# "max_iter" funciona como criterio de parada: le dice que itere un máximo de veces, acá 10000 veces 
# que itere es que repite el ajuste de parametros para minimizar la funcion de perdida

lr_model = LogisticRegression(random_state=42)

lr_model, lr_pred, lr_accuracy, lr_cross_val_mean = evaluate_model(lr_model, X_train, X_test, y_train, y_test)

results['Regresion logistica'] = (lr_accuracy, lr_cross_val_mean)

c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 99 383]
 [157 569]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5529801324503312
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5135105185493071
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5529801324503312
El F1 muestra la media armónica de la precisión y el recall:
0.5146372459233279
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
4248     1.0     1.0      0.903134                  0.0
3750     1.0     0.0      0.453332                  1.0
292      1.0     1.0      0.575562                  0.0
2895     1.0     1.0      0.613062                  0.0
668      0.0     1.0      0.673772                  1.

c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNN

In [38]:
knn_model = KNeighborsClassifier()

knn_model, knn_pred, knn_accuracy, knn_cross_val_mean = evaluate_model(knn_model, X_train, X_test, y_train, y_test)

results['KNN'] = (knn_accuracy, knn_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[142 340]
 [245 481]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.515728476821192
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.49851005518774216
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.515728476821192
El F1 muestra la media armónica de la precisión y el recall:
0.5041272973573399
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
4248     1.0     0.0           0.4                  1.0
3750     1.0     0.0           0.4                  1.0
292      1.0     0.0           0.4                  1.0
2895     1.0     0.0           0.4                  1.0
668      0.0     1.0           0.8                  1.0

# 9) Ensamblo los modelos

9.a) Defino el ensamble y ejecuto la función que acabo de crear para dicho ensamble

In [39]:
# los ensambles buscan aprovechar las capacidades predictivas de los modelos ensamblados.
# si, por caso, el arbol de decision viene modelando bien a los desertores y el xgboost viene modelando bien
# a los no desertores, entonces se predecidrá un cacho con un modelo y otro cacho con el otro modelo.

voting_clf = VotingClassifier(estimators=[ #con "estimator" se pasa una lista con los modelos ya entrenados, a los que anteponemos
    ('dt', dt_model),                      # un nombre de fantasia            
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lr', lr_model),
    ('knn', knn_model)
], voting='soft') #con "voting" designamos el tipo de votacion:
                  # 1) "soft" --> combino las probabilidades de cada uno --> esta suele ser la mejor opción
                  # 2) "hard" --> votacion simple

voting_clf, voting_pred, voting_accuracy, voting_cross_val_mean = evaluate_model(voting_clf, X_train, X_test, y_train, y_test)

results['Ensamble voting'] = (voting_accuracy, voting_cross_val_mean)

c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[141 341]
 [195 531]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5562913907284768
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5334121637749213
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5562913907284768
El F1 muestra la media armónica de la precisión y el recall:
0.5369634653786962
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
4248     1.0     1.0      0.578292                  0.0
3750     1.0     0.0      0.345111                  1.0
292      1.0     1.0      0.667138                  0.0
2895     1.0     0.0      0.453185                  1.0
668      0.0     1.0      0.733663                  1.

# 10) Genero una vista resumen de algunas de las métricas de evaluación de los modelos

In [40]:
# en las vistas anteriores se podían ver muchas métricas de evaluación.
# acá dejamos el accuracy y el promedio de las validaciones para tener una mirada rápida.

vista_resultados = pd.DataFrame.from_dict(results, orient='index', columns=['Accuracy', 'Cross Val Mean'])

vista_resultados.reset_index(inplace=True)

vista_resultados.columns = ['Model', 'Accuracy', 'Cross Val Mean']

print(vista_resultados)

                 Model  Accuracy  Cross Val Mean
0    Arbol de decision  0.519040        0.518638
1        Random Forest  0.586093        0.586221
2             XG Boost  0.556291        0.554749
3  Regresion logistica  0.552980        0.555742
4                  KNN  0.515728        0.534709
5      Ensamble voting  0.556291        0.558228


<span style="color:red">Segundo modelado</span>

Volvemos a data_1 que llega hasta el paso 4.b inclusive.

Vamos a avanzar continuando con el procesamiento de ese data frame.

Por repetir los pasos del "primer modelado", repetimos la numeración de los pasos anteriores pero agregandole la palabra ".bis". Por caso, "4.bis"

# 4.bis) Preliminar EDA y Feature Selection y Feature Engineering

<span style="color:blue">Para el procesamiento de variables que se puede ver en esta sección recurrimos a dos modalidades que adaptamos según el caso:

<span style="color:blue">O creamos una función y luego la ejecutamos mediante: data_1['nueva_variable'] = data_1.apply(funcion_creada, axis=1)

<span style="color:blue">O recurrimos a lambda tal que: 
                            
                            data_1['nueva_variable'] = data_1.apply(
                                 lambda ...
                                 else ...
                                 axis=1) 

<span style="color:blue">Al margen de que por estos medios alcanzamos el resultado deseado, ¿son estas las formas más recomendadas? ¿O se recomienda otro tipo de procedimiento? </span>

4.a.bis) Renombro y recategorizo variables 

Retomamos hasta antes de sustiuir valores nulos porque nos valdremos muchas de las variables que terminan por tener valores sustituidos para hacer algunas recategorizaciones. Sucede que algunos valores nulos son consecuencia de genuinos pases en el formulario que dio lugar a muchas de nuestras variables.

De la propia recategorización llegaremos a eliminar variables que ya quedan contenidas en variables nuevas. Sólo después de eso tomaremos acciones con respecto a los valores nulos que aun persistan.

In [41]:
nivel_conoc = []

for col in data_1.columns:
    if col.startswith('nivel_conoc'):
        nivel_conoc.append(col)    

for col in nivel_conoc:
  data_1[col] = data_1[col].replace({'1. Ningún conocimiento': 1,
                                     '2. Poco conocimiento': 2,
                                     '3. Algún conocimiento': 3,
                                     '4. Mucho conocimiento': 4})

data_1['r_gradiente_conocimiento_carrera'] = data_1.apply(
    lambda x: np.nan if x[nivel_conoc].isnull().any() 
    else x[nivel_conoc].sum()/6*2.5,
    axis=1
)

print(data_1.shape)

data_1[nivel_conoc + ['r_gradiente_conocimiento_carrera']].sample(n=10)


C:\Users\facundo.troyer\AppData\Local\Temp\ipykernel_8840\2402749881.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_1[col] = data_1[col].replace({'1. Ningún conocimiento': 1,


(6037, 154)


,nivel_conocimiento_materias_plan_estudio,nivel_conocimiento_duracion_carrera,nivel_conocimiento_funciones_graduado,nivel_conocimiento_mercado_laboral,nivel_conocimiento_salario_docente,nivel_conocimiento_estatuto_docente,r_gradiente_conocimiento_carrera
4225,2.0,3.0,3.0,3.0,1.0,1.0,5.416667
389,4.0,4.0,NaN,NaN,NaN,NaN,NaN
3939,2.0,4.0,2.0,1.0,1.0,1.0,4.583333
3295,3.0,4.0,3.0,3.0,4.0,3.0,8.333333
763,1.0,2.0,NaN,3.0,2.0,2.0,NaN
595,4.0,3.0,4.0,3.0,3.0,3.0,8.333333
1702,4.0,4.0,3.0,2.0,1.0,1.0,6.250000
3213,3.0,3.0,3.0,3.0,2.0,1.0,6.250000
1389,2.0,3.0,2.0,NaN,NaN,NaN,NaN
4382,3.0,3.0,3.0,3.0,3.0,3.0,7.500000


In [42]:
data_1 = data_1.drop(columns=nivel_conoc)

print(data_1.shape)

(6037, 148)


--------

In [43]:
mot_propuesta = []

for col in data_1.columns:
    if col.startswith('mot_prop'):
        mot_propuesta.append(col)

def r_mot_propuesta(var):
    for mot in mot_propuesta:
        if var[mot] == 1.0:
            return mot
    return np.nan

data_1['r_mot_propuesta_principal'] = data_1.apply(r_mot_propuesta, axis=1)

print(data_1.shape)

data_1[mot_propuesta + ['r_mot_propuesta_principal']].sample(n=10)

(6037, 149)


,mot_prop_sugerencia_nucleo_familiar,mot_prop_interes_ejercer_docencia,mot_prop_gusto_trabajo_niños,mot_prop_opcion_laboral_estable,mot_prop_remuneracion_docente,mot_prop_interes_area_disciplinar,mot_prop_asigno_valor_conocimiento,mot_prop_prestigio_social_profesion,mot_prop_permite_trabajar_act_personales,mot_prop_no_sabia_que_estudiar,r_mot_propuesta_principal
3962,6.0,8.0,1.0,3.0,5.0,9.0,4.0,7.0,2.0,10.0,mot_prop_gusto_trabajo_niños
1728,2.0,9.0,6.0,1.0,8.0,3.0,7.0,4.0,5.0,10.0,mot_prop_opcion_laboral_estable
231,9.0,1.0,2.0,6.0,8.0,3.0,4.0,7.0,5.0,10.0,mot_prop_interes_ejercer_docencia
4704,7.0,10.0,8.0,6.0,9.0,5.0,4.0,3.0,2.0,1.0,mot_prop_no_sabia_que_estudiar
4146,10.0,9.0,8.0,7.0,5.0,4.0,6.0,3.0,2.0,1.0,mot_prop_no_sabia_que_estudiar
4909,4.0,9.0,8.0,3.0,2.0,7.0,10.0,1.0,5.0,6.0,mot_prop_prestigio_social_profesion
1772,3.0,8.0,9.0,6.0,7.0,2.0,1.0,4.0,5.0,10.0,mot_prop_asigno_valor_conocimiento
2090,6.0,5.0,9.0,1.0,8.0,7.0,2.0,10.0,3.0,4.0,mot_prop_opcion_laboral_estable
3182,5.0,7.0,9.0,6.0,4.0,3.0,8.0,2.0,1.0,10.0,mot_prop_permite_trabajar_act_personales
5081,2.0,1.0,5.0,3.0,7.0,4.0,6.0,9.0,10.0,8.0,mot_prop_interes_ejercer_docencia


In [44]:
data_1 = data_1.drop(columns=mot_propuesta)

print(data_1.shape)

(6037, 139)


-----

In [45]:
print(data_1['ult_est_cur_padre'].unique())

['Colegio Secundario incompleto' 'Escuela primaria incompleta'
 'Colegio Secundario completo' 'Escuela primaria completa'
 'Est. universitarios completos' 'Desconoce' 'Est. superiores incompletos'
 'Est. universitarios incompletos' 'No hizo estudios'
 'Est. superiores completos' 'Est. de Post grado']


In [46]:
# Dejo "Est. superiores completos" y "Est. universitarios completos" bajo el mismo nombre
# Lo mismo con los incompletos

ult_padre_madre = ['ult_est_cur_padre', 'ult_est_cur_madre']

for col in ult_padre_madre:
  data_1[col] = data_1[col].replace({'Est. universitarios completos': 'Est. superiores completos',
                                     'Est. universitarios incompletos': 'Est. superiores incompletos'})

jerarquia_est_padres = ['Est. de Post grado',
                       'Est. superiores completos',
                       'Est. superiores incompletos',
                       'Colegio Secundario completo',
                       'Colegio Secundario incompleto',
                       'Escuela primaria completa',
                       'Escuela primaria incompleta',
                       'No hizo estudios',
                       'Desconoce']

def creo_r_ult(var):
  for abc in jerarquia_est_padres:
     for col in ult_padre_madre: 
      if var[col] == abc:
        return abc

data_1['r_ult_est_cur_padremadre'] = data_1.apply(creo_r_ult, axis=1)

print(data_1.shape)

data_1[ult_padre_madre + ['r_ult_est_cur_padremadre']].sample(n=10)

(6037, 140)


,ult_est_cur_padre,ult_est_cur_madre,r_ult_est_cur_padremadre
2434,Est. superiores incompletos,Est. superiores completos,Est. superiores completos
414,Colegio Secundario completo,Est. superiores completos,Est. superiores completos
5561,Est. superiores incompletos,Colegio Secundario completo,Est. superiores incompletos
3405,Est. superiores incompletos,Est. superiores completos,Est. superiores completos
387,Desconoce,Desconoce,Desconoce
2663,Colegio Secundario completo,Est. superiores incompletos,Est. superiores incompletos
448,Est. superiores completos,Colegio Secundario completo,Est. superiores completos
4723,Est. superiores completos,Est. superiores completos,Est. superiores completos
64,Escuela primaria completa,Colegio Secundario incompleto,Colegio Secundario incompleto
2647,Colegio Secundario incompleto,Colegio Secundario incompleto,Colegio Secundario incompleto


In [47]:
data_1 = data_1.drop(columns=ult_padre_madre)

print(data_1.shape)

(6037, 138)


---

In [48]:
data_1.sit_laboral_padre.unique()

array(['Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)',
       nan,
       'No trabajó y no buscó trabajo (no esta pensando en trabajar)',
       'Desconoce',
       'No trabajó y buscó trabajo en algún momento de los últimos 30 días'],
      dtype=object)

In [49]:
trab_padre_madre = ['sit_laboral_padre', 'sit_laboral_madre']

jerarquia_trab_padres = ['Trabajó al menos una hora (incluye a los que no trabajaron por licencia, vacaciones, enfermedad)',
                       'No trabajó y buscó trabajo en algún momento de los últimos 30 días',
                       'No trabajó y no buscó trabajo (no esta pensando en trabajar)',           
                       'Desconoce']

def creo_r_trab_padremadre(var):
  for abc in jerarquia_trab_padres:
     for col in trab_padre_madre: 
      if var[col] == abc:
        return abc
  return np.nan

data_1['r_trab_padremadre'] = data_1.apply(creo_r_trab_padremadre, axis=1)

print(data_1.shape)

data_1[trab_padre_madre + ['r_trab_padremadre']].sample(n=10)

(6037, 139)


,sit_laboral_padre,sit_laboral_madre,r_trab_padremadre
4913,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...
3146,NaN,NaN,NaN
4567,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...
4977,NaN,NaN,NaN
3159,No trabajó y no buscó trabajo (no esta pensand...,No trabajó y no buscó trabajo (no esta pensand...,No trabajó y no buscó trabajo (no esta pensand...
2615,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...
4473,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...
1671,Trabajó al menos una hora (incluye a los que n...,No trabajó y buscó trabajo en algún momento de...,Trabajó al menos una hora (incluye a los que n...
2602,NaN,NaN,NaN
5571,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...,Trabajó al menos una hora (incluye a los que n...


In [50]:
data_1 = data_1.drop(columns=trab_padre_madre)

print(data_1.shape)

(6037, 137)


----

In [51]:
data_1['estas_finalizando_secu_terminalidad'] = data_1.apply(
    lambda x: np.nan if pd.isna(x['estas_finalizando_secu_terminalidad'])
    else 'No' if x['estas_finalizando_secu_terminalidad'] == 'No'
    else 'Si',
    axis=1
)

-----

In [52]:
alu_otestsup = []

for col in data_1.columns:
    if col.startswith('alu_otestsup'):
        alu_otestsup.append(col)

alu_otestsup

['alu_otestsup_area4',
 'alu_otestsup_niv4',
 'alu_otestsup_estado4',
 'alu_otestsup_razon_abandono4',
 'alu_otestsup_area1',
 'alu_otestsup_niv1',
 'alu_otestsup_estado1',
 'alu_otestsup_area2',
 'alu_otestsup_niv2',
 'alu_otestsup_estado2',
 'alu_otestsup_razon_abandono2']

In [53]:
estudios_previos = []

for col in data_1.columns:
    if col.startswith('alu_otestsup_estado'):
        estudios_previos.append(col)

jerarquia_est_previo = ['Finalizado',
                        'En curso',
                        'Abandonado']

def creo_r_mayor(var):
  for abc in jerarquia_est_previo:
    for col in estudios_previos:
      if var[col] == abc:
        return abc
  return np.nan

data_1['r_mayor_estudio_alcanzado'] = data_1.apply(creo_r_mayor, axis=1)

data_1[estudios_previos + ['r_mayor_estudio_alcanzado']].sample(n=10)

,alu_otestsup_estado4,alu_otestsup_estado1,alu_otestsup_estado2,r_mayor_estudio_alcanzado
5711,NaN,NaN,NaN,NaN
130,NaN,NaN,NaN,NaN
5027,NaN,NaN,NaN,NaN
2763,Abandonado,NaN,NaN,Abandonado
1848,NaN,NaN,NaN,NaN
3986,En curso,NaN,NaN,En curso
5289,Abandonado,NaN,NaN,Abandonado
5737,NaN,NaN,NaN,NaN
3544,NaN,NaN,NaN,NaN
4459,NaN,NaN,NaN,NaN


----

In [54]:
numerador = [4, 1, 2]

def r_mayor_nivel(var):
    for num in numerador:
        if var['alu_otestsup_estado' + str(num)] == var['r_mayor_estudio_alcanzado']:
            return var['alu_otestsup_niv' + str(num)]
    return np.nan

data_1['r_mayor_estudio_alcanzado_nivel'] = data_1.apply(r_mayor_nivel, axis=1)

estudios_previos_nivel = []

for col in data_1.columns:
    if col.startswith('alu_otestsup_niv'):
        estudios_previos_nivel.append(col)

data_1[estudios_previos + estudios_previos_nivel + ['r_mayor_estudio_alcanzado', 'r_mayor_estudio_alcanzado_nivel']].sample(n=10)

,alu_otestsup_estado4,alu_otestsup_estado1,alu_otestsup_estado2,alu_otestsup_niv4,alu_otestsup_niv1,alu_otestsup_niv2,r_mayor_estudio_alcanzado,r_mayor_estudio_alcanzado_nivel
2120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,Abandonado,En curso,NaN,Licenciatura,Tecnicatura,NaN,En curso,Tecnicatura
4911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3671,Abandonado,NaN,NaN,Tecnicatura,NaN,NaN,Abandonado,Tecnicatura
2620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


____

In [55]:
# Las columnas relativas al área, la carrera, el año de egreso y el motivo de abandonos del estudios superior previo las eliminamos sin más

estudios_previos_lista = []

estudios_previos_elimino = ['alu_otestsup_area',
                            'alu_otestsup_razon_abandono']

for col in data_1.columns:
    for elim in estudios_previos_elimino:
         if col.startswith(elim):
             estudios_previos_lista.append(col)

print(data_1.shape)

columnas_a_eliminar_estudio = estudios_previos + estudios_previos_nivel + estudios_previos_lista

print(columnas_a_eliminar_estudio)

data_1 = data_1.drop(columns=columnas_a_eliminar_estudio)

print(data_1.shape)

(6037, 139)
['alu_otestsup_estado4', 'alu_otestsup_estado1', 'alu_otestsup_estado2', 'alu_otestsup_niv4', 'alu_otestsup_niv1', 'alu_otestsup_niv2', 'alu_otestsup_area4', 'alu_otestsup_razon_abandono4', 'alu_otestsup_area1', 'alu_otestsup_area2', 'alu_otestsup_razon_abandono2']
(6037, 128)


----

In [56]:
medios_comu = []

for col in data_1.columns:
    if col.startswith('medios_comunicacion') and col != 'medios_comunicacion_regularidad':
        medios_comu.append(col)

data_1['r_cantidad_medios_comunicacion'] = data_1.apply(
    lambda x: np.nan if x[medios_comu].isna().all()
    else sum(x[medios_comu] == 'Si'), 
    axis=1)

print(data_1.shape)

data_1[medios_comu + ['r_cantidad_medios_comunicacion']].sample(n=10)

(6037, 129)


,medios_comunicacion_tv,medios_comunicacion_web,medios_comunicacion_periodico_impreso,medios_comunicacion_app,medios_comunicacion_podcast,medios_comunicacion_radio,medios_comunicacion_redes_sociales,medios_comunicacion_revista_impresa,medios_comunicacion_boletin_noticias,medios_comunicacion_otro,r_cantidad_medios_comunicacion
4520,Si,Si,No,Si,No,No,No,No,No,No,3.0
5063,No,No,No,No,No,No,No,No,No,No,0.0
3675,No,No,Si,Si,No,No,No,No,Si,No,3.0
5170,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5351,Si,No,Si,Si,No,No,No,No,No,No,3.0
4007,Si,Si,No,Si,No,No,No,No,No,No,3.0
2906,Si,No,No,No,No,No,No,No,No,No,1.0
49,Si,No,No,Si,No,No,No,No,No,No,2.0
5416,Si,No,No,Si,No,No,No,No,No,No,2.0
1518,No,No,No,No,No,No,No,No,No,No,0.0


In [57]:
data_1 = data_1.drop(columns=medios_comu)

print(data_1.shape)

(6037, 119)


----

In [58]:
# Por la forma que tiene el formulario que genera parte de estos datos al momento de indagar por estas variables, no se puede 
# asumir que los "No" sean efectivamente "Nos". Sucede que si los respondentes dejaban la pregunta en blanco, se registraba como "No".
# Como consecencia de ello, sólo es legítimo contar por la positiva. Con ello, es probable que allí donde todas las respuestas sean "No",
# se trata en realidad de alguien que eligió no responder esta sección del formulario.

actividades_culturales = []

for col in data_1.columns:
    if col.startswith('actividad_medios'):
        actividades_culturales.append(col)

data_1['r_cantidad_actividades_culturales'] = data_1.apply(
    lambda x: np.nan if (x[actividades_culturales] == 'No').all()
    else sum(x[actividades_culturales] == 'Si'), 
    axis=1)

print(data_1.shape)

data_1[actividades_culturales + ['r_cantidad_actividades_culturales']].sample(n=10)

(6037, 120)


,actividad_medios_espectaculo,actividad_medios_expo_arte,actividad_medios_leer_libros,actividad_medios_peliculas_series,actividad_medios_danza_ballet,actividad_medios_instrumento_canto,actividad_medios_actividades_sociales_politicas_barriales,actividad_medios_teatro,actividad_medios_museos,actividad_medios_cine,actividad_medios_podcast_radio,actividad_medios_club,actividad_medios_organizacion_estudiantil,r_cantidad_actividades_culturales
1307,No,No,No,No,No,Si,Si,Si,No,No,No,No,No,3.0
2770,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN
4007,Si,Si,No,Si,Si,Si,Si,Si,No,Si,Si,No,No,9.0
664,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN
1253,No,No,No,No,No,Si,No,Si,No,No,No,No,No,2.0
399,No,No,No,No,Si,No,Si,No,No,No,No,No,No,2.0
5173,No,No,No,No,Si,Si,No,No,No,Si,No,No,No,3.0
1420,No,No,No,No,No,No,No,No,No,No,No,No,No,NaN
4740,Si,No,No,No,No,Si,Si,Si,No,No,No,No,No,4.0
5718,No,No,Si,Si,No,Si,Si,No,No,No,No,No,Si,5.0


In [59]:
data_1 = data_1.drop(columns=actividades_culturales)

print(data_1.shape)

(6037, 107)


----

In [60]:
idiomas = []

for col in data_1.columns:
    if col.startswith('nivel_idioma'):
        idiomas.append(col)

data_1['r_cantidad_idiomas_buenos'] = data_1.apply(
    lambda x: np.nan if x[idiomas].isna().all() 
    else sum(x[idiomas].isin(['Muy bueno', 'Bueno'])), 
    axis=1)

print(data_1.shape)

data_1[idiomas + ['r_cantidad_idiomas_buenos']].sample(n=10)

(6037, 108)


,nivel_idioma_ingles,nivel_idioma_frances,nivel_idioma_portugues,nivel_idioma_italiano,nivel_idioma_aleman,nivel_idioma_chino,nivel_idioma_otro,r_cantidad_idiomas_buenos
1116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5284,Bueno,Desconoce,Desconoce,Desconoce,Desconoce,Desconoce,NaN,1.0
5204,Basico,Desconoce,Desconoce,Desconoce,Desconoce,Desconoce,NaN,0.0
4975,Basico,Basico,Desconoce,Basico,Basico,Desconoce,NaN,0.0
3408,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2252,Muy bueno,Desconoce,Desconoce,Desconoce,Desconoce,Desconoce,NaN,1.0
5897,Muy bueno,Desconoce,Desconoce,Desconoce,Desconoce,Desconoce,NaN,1.0
538,Basico,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3293,Basico,Desconoce,Desconoce,Desconoce,Desconoce,Desconoce,NaN,0.0
1895,Bueno,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
data_1 = data_1.drop(columns=idiomas)

print(data_1.shape)

(6037, 101)


-----

In [62]:
practica_deporte = []

for col in data_1.columns:
    if col.startswith('prac_dep'):
        practica_deporte.append(col)

data_1['r_cantidad_deportes'] = data_1.apply(
    lambda x: np.nan if x[practica_deporte].isna().all() 
    else 0 if x['practica_deportes'] == 'N'
    else sum(x[practica_deporte] == 'S'), 
    axis=1
)

print(data_1.shape)

data_1[['practica_deportes'] + practica_deporte + ['r_cantidad_deportes']].sample(n=10)

(6037, 102)


,practica_deportes,prac_dep_futbol,prac_dep_basquet,prac_dep_voley,prac_dep_gimnasia,prac_dep_tenis,prac_dep_natacion,prac_dep_handball,prac_dep_otros,r_cantidad_deportes
2768,S,N,N,S,S,N,N,N,N,2.0
3965,S,N,N,N,N,S,S,S,N,3.0
3040,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
402,S,N,N,S,N,N,N,N,N,1.0
1314,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2942,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537,S,S,N,N,N,N,N,N,N,1.0
990,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1863,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4121,S,N,N,N,N,N,S,N,N,1.0


In [63]:
data_1 = data_1.drop(columns=practica_deporte + ['practica_deportes'])

print(data_1.shape)

(6037, 93)


-----

4.b.bis) Identifico aun dentro de mi data frame la variable que en un futuro va a ser mi VD y reemplazo sus valores por números

In [64]:
# esto ya quedo hecho en el primer modelado

4.c.bis) Sustituyo nulos

El proceso de sustitución de nulos al que recurriré ahora cambia al del primer modelado:

Antes identificamos nulos y los sustituimos sin más.

Ahora que nos acercamos a un data frame más definitivo vamos a partir eliminando todas las variables que tengan hasta 1500 valores nulos. Sucede que en un data frame que tiene cerca de 6000 filas sustituir valores en variables que tienen entre 1500 y 6000 valores nulos nos dan como resultado variables que son puro valor sustituido. La idea es, primero, sacar esas variables y, luego, sustituir valores.

In [65]:
nulos = data_1.isnull().sum()

columnas_a_eliminar = nulos[nulos > 1500].index

print(data_1.shape)

print(columnas_a_eliminar)

# ciertamente termina por eliminar muchas de las variables que nosotros creamos.
# las hicimos para ver si, tras unir los resultados de varias columnas en uno, lograbamos que baje la cantidad de nulos.
# se ve que no sucedió tanto como hubiesemos querido 

(6037, 93)
Index(['alu_cos_est_tiene_plan_social', 'alu_cos_est_tiene_trabajo',
       'alu_cos_est_tiene_beca', 'alu_trab_hace', 'alu_trab_ocup',
       'hora_sem_trab_alum', 'vinculacion_docencia', 'dias_trab_alum',
       'franja_horaria_trab_alum', 'alu_trab_desjub', 'alu_trab_fami',
       'alu_trab_remuneracion', 'padre_trab_hace', 'padre_trab_ocup',
       'padre_trab_otro', 'madre_trab_hace', 'madre_trab_ocup',
       'madre_trab_otro', 'alu_estudio_previo2', 'alu_tec_comp_escritorio',
       'alu_tec_comp_portatil', 'alu_tec_table', 'alu_tec_uso_exclusivo',
       'alu_tec_estado', 'disp_actuales_tel', 'disp_actuales_ereader',
       'disp_actuales_auriculares', 'hace_dep_gim_priv', 'hace_dep_univ',
       'hace_dep_partic', 'hace_dep_otros', 'medios_comunicacion_regularidad',
       'expectativa_rol', 'r_mayor_estudio_alcanzado',
       'r_mayor_estudio_alcanzado_nivel', 'r_cantidad_medios_comunicacion',
       'r_cantidad_actividades_culturales', 'r_cantidad_deportes'],
    

In [66]:
data_1 = data_1.drop(columns=columnas_a_eliminar)

print(data_1.shape)

(6037, 55)


In [67]:
# Además, reviso si no tengo alguna columna con todos valores nulos
# De existir la dropeo

variables_nulas = data_1.columns[data_1.isnull().all()]

print(variables_nulas)

print(data_1.shape)

data_1 = data_1.drop(columns=variables_nulas)

print(data_1.shape)

Index([], dtype='object')
(6037, 55)
(6037, 55)


In [68]:
# ahora vuelvo a crear mi data frame "data_1_imputado", pero tras todo el procesamiento y la eliminación de variables con muchos valores nulos

data_1_imputado = data_1.copy()

variables_categoricas_im = data_1.select_dtypes(include=['object', 'category']).columns

variables_numericas_im = data_1.select_dtypes(include=['int64', 'float64']).columns

imputador_categorico = SimpleImputer(strategy='most_frequent')  

imputador_numerico = SimpleImputer(strategy='mean')

data_1_imputado[variables_categoricas_im] = imputador_categorico.fit_transform(data_1[variables_categoricas_im])

data_1_imputado[variables_numericas_im] = imputador_numerico.fit_transform(data_1[variables_numericas_im])

print(data_1.shape)

print(data_1_imputado.shape)

nulos = data_1.isnull().sum()

print(nulos.sort_values(ascending=False))

(6037, 55)
(6037, 55)
expectativa_lugar_de_trabajo           1437
alu_tec_calidad_conec                  1321
r_cantidad_idiomas_buenos              1314
alu_tec_espacio_fisico                 1292
alu_tec_pc_casa_caba                   1281
alu_tec_tipo_conec                     1274
estas_finalizando_secu_terminalidad    1144
tipo_gestion_instit_fin_secu           1047
cuantos_medios_transportes_insume      1035
r_trab_padremadre                       869
r_gradiente_conocimiento_carrera        589
cuanto_tiempo_insumira_viajes           572
familiar_ejerce_docencia                449
vive_actual_con                         344
alu_estudio_previo4                      87
alu_estudio_previo1                      87
r_mot_propuesta_principal                85
obra_social_alu                           2
alu_cos_est_ahorro                        0
madre_vive                                0
padre_vive                                0
cant_fami_cargo                           0
cantidad_h

In [69]:
nulos = data_1_imputado.isnull().sum()

print(nulos.sort_values(ascending=False))

genero                                 0
alu_estudio_previo1                    0
alu_cos_est_ahorro                     0
alu_cos_est_otra                       0
existe_trab_alum                       0
estado_civil                           0
cantidad_hijos                         0
cant_fami_cargo                        0
padre_vive                             0
madre_vive                             0
estas_finalizando_secu_terminalidad    0
tipo_gestion_instit_fin_secu           0
alu_estudio_previo4                    0
alu_tec_pc_casa_caba                   0
alu_cos_est_trab                       0
alu_tec_espacio_fisico                 0
alu_tec_tipo_conec                     0
alu_tec_calidad_conec                  0
expectativa_lugar_de_trabajo           0
obra_social_alu                        0
r_edad                                 0
r_desercion_temprana                   0
r_gradiente_conocimiento_carrera       0
r_mot_propuesta_principal              0
r_ult_est_cur_pa

4.d.bis) Elimino duplicados

In [70]:
data_1_imputado.duplicated().sum()

27

<span style="color:blue"> ¿Eliminar filas duplicadas es adecuado? ¿Siempre debe hacerse o sólo algunas veces?

In [71]:
# se ve que tras la reducción de variables, aparecen las filas duplicadas.
# procedemos a eliminarlas

data_1_imputado = data_1_imputado.drop_duplicates()

print(data_1_imputado.shape)

(6010, 55)


# 5.bis) Identifico aun dentro de mi data frame las variables que en un futuro van a ser mis VIs y:

In [72]:
variables_independientes = data_1_imputado.columns.drop(['r_desercion_temprana'])
variables_independientes

Index(['genero', 'familiar_ejerce_docencia', 'mot_inst_publica',
       'mot_inst_difusion', 'mot_inst_rec_amigos', 'mot_inst_oferta_unica',
       'mot_inst_plan_estudio', 'mot_inst_prestigio', 'mot_inst_rec_estud',
       'mot_inst_ubicacion', 'mot_inst_turno', 'mot_inst_conozco_inst',
       'mot_inst_sist_ing', 'mot_inst_otros', 'desafio_tareas',
       'desafio_falta_recursos', 'desafio_dif_horarios', 'desafio_ns_nc',
       'desafio_dif_teoricos', 'desafio_superposicion', 'desafio_incer_fin',
       'desafio_otra', 'tipo_vivienda', 'vive_actual_con',
       'cuanto_tiempo_insumira_viajes', 'cuantos_medios_transportes_insume',
       'alu_cos_est_ap_fam', 'alu_cos_est_plsoc', 'alu_cos_est_trab',
       'alu_cos_est_beca', 'alu_cos_est_ahorro', 'alu_cos_est_otra',
       'existe_trab_alum', 'estado_civil', 'cantidad_hijos', 'cant_fami_cargo',
       'padre_vive', 'madre_vive', 'estas_finalizando_secu_terminalidad',
       'tipo_gestion_instit_fin_secu', 'alu_estudio_previo4',
     

5.a.bis) Distingo aun dentro de mi data frame VIs categóricas y VIs numéricas

In [73]:
# a traves de ".columns" me quedo sólo con el nombre de las columnas, de lo contrario me devuelve una salida tipo data frame

variables_categoricas = data_1_imputado[variables_independientes].select_dtypes(include=['object', 'category']).columns
variables_categoricas

Index(['genero', 'familiar_ejerce_docencia', 'mot_inst_publica',
       'mot_inst_difusion', 'mot_inst_rec_amigos', 'mot_inst_oferta_unica',
       'mot_inst_plan_estudio', 'mot_inst_prestigio', 'mot_inst_rec_estud',
       'mot_inst_ubicacion', 'mot_inst_turno', 'mot_inst_conozco_inst',
       'mot_inst_sist_ing', 'mot_inst_otros', 'desafio_tareas',
       'desafio_falta_recursos', 'desafio_dif_horarios', 'desafio_ns_nc',
       'desafio_dif_teoricos', 'desafio_superposicion', 'desafio_incer_fin',
       'desafio_otra', 'tipo_vivienda', 'vive_actual_con',
       'cuanto_tiempo_insumira_viajes', 'alu_cos_est_ap_fam',
       'alu_cos_est_plsoc', 'alu_cos_est_trab', 'alu_cos_est_beca',
       'alu_cos_est_ahorro', 'alu_cos_est_otra', 'existe_trab_alum',
       'estado_civil', 'cantidad_hijos', 'cant_fami_cargo', 'padre_vive',
       'madre_vive', 'estas_finalizando_secu_terminalidad',
       'tipo_gestion_instit_fin_secu', 'alu_estudio_previo4',
       'alu_estudio_previo1', 'alu_tec_pc_

In [74]:
variables_numericas = data_1_imputado[variables_independientes].select_dtypes(include=[np.int32, np.int64, np.float64]).columns
variables_numericas

Index(['cuantos_medios_transportes_insume', 'r_edad',
       'r_gradiente_conocimiento_carrera', 'r_cantidad_idiomas_buenos'],
      dtype='object')

5.b.bis) Transformo aun dentro de mi data frame VIs categóricas a dummies

In [75]:
# dummies es la forma más tradicional de transformar variables categóricas a numéricas, por medio
# de crear una columna nueva por cada valor de la variable categórica.
# trabajo   condicion_trabajo
#   si           monotriburo
#   si           monotributo
#   si           relacion de dependencia
#   no              -

#  trabajo_si   trabajo_no   condicion_trabajo_monotributo     condicion_trabajo_relacion_dependencia
#     True         False              True                              False
#     True         False              True                              False
#     True         False              False                             True
#     False        False              False                             False

data_1_imputado = pd.get_dummies(data_1_imputado, columns=variables_categoricas, drop_first=True)

print(data_1_imputado.shape)

(6010, 106)


5.c.bis) Transformo aun dentro de mi data frame VIs numéricas a VIs numéricas escaladas

In [76]:
# ponderar sirve para que el modelo no se vea afectado por variables que, por su propia naturaleza, tienen valores mas altos (puesto 
# que podria ponderar mas esas variables)
# y sirve para no verse afectado por la distorción/ el desvio de las variables

# REVISAR ESTAS 2 AFIRMACIONES:
# parece que estandatiza entre -3 y 3
# Esto asegura que cada característica tenga una media de 0 y una desviación estándar de 1

scaler = StandardScaler()
data_1_imputado[variables_numericas] = scaler.fit_transform(data_1_imputado[variables_numericas])

print("Seguimos ante un conjunto de variables que a su interior tienen series")
print(type(data_1_imputado[variables_numericas]))
print(type(data_1_imputado[variables_numericas[0]]))
print("Vemos el cambio de valores dentro de las variales")
print(data[variables_numericas[0]])
print(data_1_imputado[variables_numericas[0]])
print("Vemos el cambio de promedios")
print(np.mean (data[variables_numericas[0]]))
print(np.mean (data_1_imputado[variables_numericas[0]]))

Seguimos ante un conjunto de variables que a su interior tienen series
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
Vemos el cambio de valores dentro de las variales
0       2.0
1       2.0
2       NaN
3       1.0
4       1.0
       ... 
6032    1.0
6033    1.0
6034    1.0
6035    2.0
6036    NaN
Name: cuantos_medios_transportes_insume, Length: 6037, dtype: float64
0       0.311839
1       0.311839
2      -0.000454
3      -0.966468
4      -0.966468
          ...   
6032   -0.966468
6033   -0.966468
6034   -0.966468
6035    0.311839
6036   -0.000454
Name: cuantos_medios_transportes_insume, Length: 6010, dtype: float64
Vemos el cambio de promedios
1.7556977209116353
-1.513302332400879e-16


# 6.bis) Salgo de mi data frame y creo mis objetos nuevos y e X

In [77]:
# "axis=1" 
# la función de "axis=1" es dar cuenta que hacemos referencia a una columna.
# "axis=0", mientras tanto, da cuenta de que hablamos de una fila.
#
# en "y = data_1['Target']" no hace falta el "axis=1" porque esa enunciación es específica para seleccionar columnas
# en "X = data_1.drop('Target', axis=1)" se trabaja sobre un data frame y el método "drop" puede operar quitando filas o columnas,
# de allí que sea necesario especificar que nuestro deseo es quitar una columna mediante "axis=1" 

y = data_1_imputado['r_desercion_temprana']
X = data_1_imputado.drop('r_desercion_temprana', axis=1)

print("El tipo de data de y que almacena una lista con los valores de una sola variable es distinto del tipo de dato de X que almacena distintas variables")
print(type(y))
print(type(X))
print(y)
print(X.columns)

# tanto la division entre y e X, como la futura division entre train y test vale para modelos supervisados, no así para no supervisados

El tipo de data de y que almacena una lista con los valores de una sola variable es distinto del tipo de dato de X que almacena distintas variables
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6032    1.0
6033    1.0
6034    1.0
6035    0.0
6036    0.0
Name: r_desercion_temprana, Length: 6010, dtype: float64
Index(['cuantos_medios_transportes_insume', 'r_edad',
       'r_gradiente_conocimiento_carrera', 'r_cantidad_idiomas_buenos',
       'genero_Masculino', 'familiar_ejerce_docencia_Si', 'mot_inst_publica_S',
       'mot_inst_difusion_S', 'mot_inst_rec_amigos_S',
       'mot_inst_oferta_unica_S',
       ...
       'r_ult_est_cur_padremadre_Desconoce',
       'r_ult_est_cur_padremadre_Escuela primaria completa',
       'r_ult_est_cur_padremadre_Escuela primaria incompleta',
       'r_ult_est_cur_padremadre_Est. de Post grado',
       'r_ult_est_cur_padremadre_Est. superiores completos',

# 7.bis) Hago el split train-test con mis objetos y e X

In [78]:
# con "stratify=y" le pedimos que conserve entre train y test la proporcion de registros con los distintos valores de y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91, stratify=y)

print("Vemos que los train tienen más filas que los test, pero tienen la misma cantidad de columnas")
print(y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(X_test.shape)
print("También vemos que y e X conservan su tipo de dato")
print(type(y_train))
print(type(y_test))
print(type(X_train))
print(type(X_test))

Vemos que los train tienen más filas que los test, pero tienen la misma cantidad de columnas
(4808,)
(1202,)
(4808, 105)
(1202, 105)
También vemos que y e X conservan su tipo de dato
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [79]:
# confirmo que sigo sin valores nulos

nulos_train = X_train.isnull().sum()
nulos_test = X_test.isnull().sum()

print("Valores nulos en X_train:")
print(nulos_train.sort_values(ascending=False))

print("Valores nulos en X_test:")
print(nulos_test.sort_values(ascending=False))

Valores nulos en X_train:
cuantos_medios_transportes_insume                                                                                     0
cant_fami_cargo_2. Dos                                                                                                0
expectativa_lugar_de_trabajo_2. Privado                                                                               0
alu_tec_calidad_conec_Regular                                                                                         0
alu_tec_calidad_conec_Muy mala                                                                                        0
                                                                                                                     ..
vive_actual_con_Otros                                                                                                 0
vive_actual_con_Con su pareja/hijos                                                                                   0
vive_actual_co

# 8.bis) Hago el modelado inicial

8.a.bis) Creo una función que contiene el diseño general de: el modelo, su entrenamiento, su predicción y su evaluación, para luego ir ejecutandolo con cada uno de los modelos que necesito

In [80]:
# no hace falta, reutilizamos la función que creamos antes

8.b) Defino los modelos y ejecuto la función que acabo de crear para cada uno de los modelos definidos

Árbol de decisión

In [81]:
results_2 = {}

In [82]:
# 1) Defino el modelo POR FUERA DE LA FUNCION:

dt_model = DecisionTreeClassifier(random_state=42)

dt_model, dt_pred, dt_accuracy, dt_cross_val_mean = evaluate_model(dt_model, X_train, X_test, y_train, y_test)

results_2['Arbol de decision 2'] = (dt_accuracy, dt_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[197 282]
 [310 413]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5074875207986689
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5122788390205635
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5074875207986689
El F1 muestra la media armónica de la precisión y el recall:
0.5096179930192132
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     0.0           0.0                  1.0
2807     0.0     0.0           0.0                  0.0
4336     1.0     0.0           0.0                  1.0
1681     1.0     1.0           1.0                  0.0
1881     1.0     0.0           0.0                  1.

Random Forest

In [83]:
rf_model = RandomForestClassifier(random_state=42)    # con "class_weight = 'balance'" se están balanceado los valores de y
                                                                               # si tengo 95% que deserto y 5% que siguio, un modelo que dice que todos
                                                                               # dejaron andaria bastante bien. 
                                                                               # para evitar eso este hiperparametro asigna más peso o, lo que se lo mismo,
                                                                               # pondera cuando la prediccion la pega en el valor de y que es poco común.
                                                                               # (minimiza la funcion de pérdida cuando se predice correctamente esos casos)
f_model, rf_pred, rf_accuracy, rf_cross_val_mean = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

results_2['Random Forest 2'] = (rf_accuracy, rf_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 50 429]
 [100 623]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.559900166389351
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.4890441891313379
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.559900166389351
El F1 muestra la media armónica de la precisión y el recall:
0.48558923212501137
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     0.0      0.500000                  1.0
2807     0.0     0.0      0.490000                  0.0
4336     1.0     1.0      0.510000                  0.0
1681     1.0     1.0      0.510000                  0.0
1881     1.0     1.0      0.550000                  0.0

XG Boost

In [84]:
xgb_model = XGBClassifier(random_state=42) 

xgb_model, xgb_pred, xgb_accuracy, xgb_cross_val_mean = evaluate_model(xgb_model, X_train, X_test, y_train, y_test)

results_2['XG Boost 2'] = (xgb_accuracy, xgb_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[137 342]
 [207 516]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.543261231281198
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5204456935897751
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.543261231281198
El F1 muestra la media armónica de la precisión y el recall:
0.5253011147535243
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0       1      0.891578                  0.0
2807     0.0       0      0.455035                  0.0
4336     1.0       0      0.319422                  1.0
1681     1.0       1      0.667532                  0.0
1881     1.0       1      0.712064                  0.0


Regresión Logística

In [85]:
# "max_iter" funciona como criterio de parada: le dice que itere un máximo de veces, acá 10000 veces 
# que itere es que repite el ajuste de parametros para minimizar la funcion de perdida

lr_model = LogisticRegression(random_state=42)

lr_model, lr_pred, lr_accuracy, lr_cross_val_mean = evaluate_model(lr_model, X_train, X_test, y_train, y_test)

results_2['Regresion logistica 2'] = (lr_accuracy, lr_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 44 435]
 [ 73 650]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5773710482529119
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5102083133858132
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5773710482529119
El F1 muestra la media armónica de la precisión y el recall:
0.4913319691389605
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.565654                  0.0
2807     0.0     1.0      0.662239                  1.0
4336     1.0     1.0      0.607581                  0.0
1681     1.0     1.0      0.584286                  0.0
1881     1.0     1.0      0.500126                  0.

KNN

In [86]:
knn_model = KNeighborsClassifier()

knn_model, knn_pred, knn_accuracy, knn_cross_val_mean = evaluate_model(knn_model, X_train, X_test, y_train, y_test)

results_2['KNN 2'] = (knn_accuracy, knn_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[158 321]
 [231 492]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.540765391014975
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5258654952358008
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.540765391014975
El F1 muestra la media armónica de la precisión y el recall:
0.5304112840925646
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0           0.6                  0.0
2807     0.0     1.0           1.0                  1.0
4336     1.0     1.0           0.6                  0.0
1681     1.0     1.0           0.8                  0.0
1881     1.0     1.0           0.6                  0.0


# 9.bis) Ensamblo los modelos

9.a.bis) Defino el ensamble y ejecuto la función que acabo de crear para dicho ensamble

In [87]:
# los ensambles buscan aprovechar las capacidades predictivas de los modelos ensamblados.
# si, por caso, el arbol de decision viene modelando bien a los desertores y el xgboost viene modelando bien
# a los no desertores, entonces se predecidrá un cacho con un modelo y otro cacho con el otro modelo.

voting_clf = VotingClassifier(estimators=[ #con "estimator" se pasa una lista con los modelos ya entrenados, a los que anteponemos
    ('dt', dt_model),                      # un nombre de fantasia            
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lr', lr_model),
    ('knn', knn_model)
], voting='soft') #con "voting" designamos el tipo de votacion:
                  # 1) "soft" --> combino las probabilidades de cada uno --> esta suele ser la mejor opción
                  # 2) "hard" --> votacion simple

voting_clf, voting_pred, voting_accuracy, voting_cross_val_mean = evaluate_model(voting_clf, X_train, X_test, y_train, y_test)

results_2['Ensamble voting 2'] = (voting_accuracy, voting_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[134 345]
 [216 507]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5332778702163061
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5105029009330526
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5332778702163061
El F1 muestra la media armónica de la precisión y el recall:
0.5160781315992674
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.511446                  0.0
2807     0.0     1.0      0.521455                  1.0
4336     1.0     0.0      0.407401                  1.0
1681     1.0     1.0      0.712364                  0.0
1881     1.0     0.0      0.472438                  1.

# 10.bis) Genero una vista resumen de algunas de las métricas de evaluación de los modelos

In [88]:
vista_resultados_2 = pd.DataFrame.from_dict(results_2, orient='index', columns=['Accuracy', 'Cross Val Mean'])

vista_resultados_2.reset_index(inplace=True)

vista_resultados_2.columns = ['Model', 'Accuracy', 'Cross Val Mean']

print(vista_resultados_2)

                   Model  Accuracy  Cross Val Mean
0    Arbol de decision 2  0.507488        0.525790
1        Random Forest 2  0.559900        0.586023
2             XG Boost 2  0.543261        0.557903
3  Regresion logistica 2  0.577371        0.586522
4                  KNN 2  0.540765        0.539101
5      Ensamble voting 2  0.533278        0.558236


10.b.bis) Junto y visualizo el accuracy y el cross val mean del primer y segundo modelado

In [89]:
vista_resultados_combined = pd.concat([vista_resultados, vista_resultados_2], ignore_index=True)

print(vista_resultados_combined)

print("""Comparando las métricas de evaluación del primer y el segundo modelado podemos ver que:
En la regresión logística y en el knn mejoré el accuracy, mientras que en el árbol, el random forest, el xgboost y el ensamble empeoré. 
Con respecto al promedio de las medidas de validación mejoré o me mantuve igual.""")

                    Model  Accuracy  Cross Val Mean
0       Arbol de decision  0.519040        0.518638
1           Random Forest  0.586093        0.586221
2                XG Boost  0.556291        0.554749
3     Regresion logistica  0.552980        0.555742
4                     KNN  0.515728        0.534709
5         Ensamble voting  0.556291        0.558228
6     Arbol de decision 2  0.507488        0.525790
7         Random Forest 2  0.559900        0.586023
8              XG Boost 2  0.543261        0.557903
9   Regresion logistica 2  0.577371        0.586522
10                  KNN 2  0.540765        0.539101
11      Ensamble voting 2  0.533278        0.558236
Comparando las métricas de evaluación del primer y el segundo modelado podemos ver que:
En la regresión logística y en el knn mejoré el accuracy, mientras que en el árbol, el random forest, el xgboost y el ensamble empeoré. 
Con respecto al promedio de las medidas de validación mejoré o me mantuve igual.


<span style="color:red">Tercer modelado</span>

<span style="color:blue">Alguno de los procesamientos en los que incurrimos antes de correr el tercer modelado estuvo mal porque si bien en la regresion linea, el random forest y el knn mejoramos un poco, empeoramos en las métricas precision y F1 en xg boost y regresión logísitica y no supimos identificar por qué.

<span style="color:blue">¿En qué nos equivocamos?</span>

# 11) Aun fuera de mi data frame podemos recurrir a distintas formas de ver qué variables independientes están pesando más en el modelo, lo que más adelante vamos a usar para ver con qué variables nos quedamos

11.a) Visualizo los modelos - sólo supe cómo visualizar el árbol de decisión -

In [90]:
# genero una lista con los nombres de las variables/ columnas que hay en el objeto "X"
# que no son los mismos que había en mi lista "variables_independientes" original porque la reduje bastante

variables_independientes_2 = X.columns.tolist() # X --> partimos de un data frame
                                   # X.columns --> pasamos a un index con los nombres de las columnas del data frame
                                   # X.columns.tolist() --> pasamos a una lista con los nombres de las columnas del data frame

print(type(X))
print(type(X.columns))
print(type(X.columns.tolist()))
print(variables_independientes_2)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.indexes.base.Index'>
<class 'list'>
['cuantos_medios_transportes_insume', 'r_edad', 'r_gradiente_conocimiento_carrera', 'r_cantidad_idiomas_buenos', 'genero_Masculino', 'familiar_ejerce_docencia_Si', 'mot_inst_publica_S', 'mot_inst_difusion_S', 'mot_inst_rec_amigos_S', 'mot_inst_oferta_unica_S', 'mot_inst_plan_estudio_S', 'mot_inst_prestigio_S', 'mot_inst_rec_estud_S', 'mot_inst_ubicacion_S', 'mot_inst_turno_S', 'mot_inst_conozco_inst_S', 'mot_inst_sist_ing_S', 'mot_inst_otros_S', 'desafio_tareas_Si', 'desafio_falta_recursos_Si', 'desafio_dif_horarios_Si', 'desafio_ns_nc_Si', 'desafio_dif_teoricos_Si', 'desafio_superposicion_Si', 'desafio_incer_fin_Si', 'desafio_otra_S', 'tipo_vivienda_Casa / Departamento propio', 'tipo_vivienda_Otros', 'tipo_vivienda_Pensión', 'tipo_vivienda_Residencia estudiantil', 'vive_actual_con_Con familia de origen (padres, hermanos, abuelos)', 'vive_actual_con_Con su pareja/hijos', 'vive_actual_con_Otros

In [91]:
# obtenengo las reglas del árbol de decisión
# ojo que los valores están escalados. esto nos sirve para ver qué variable se uso para cada ramificación, pero no para afirmar que se hizo sobre tal 
# o cuál valor

tree_rules = export_text(dt_model, feature_names=variables_independientes_2)

print(tree_rules)

|--- r_edad <= -0.72
|   |--- r_gradiente_conocimiento_carrera <= -0.24
|   |   |--- r_mot_propuesta_principal_mot_prop_remuneracion_docente <= 0.50
|   |   |   |--- expectativa_lugar_de_trabajo_4. Me imagino trabajando fuera de la educación formal <= 0.50
|   |   |   |   |--- alu_estudio_previo4_Si <= 0.50
|   |   |   |   |   |--- desafio_superposicion_Si <= 0.50
|   |   |   |   |   |   |--- desafio_incer_fin_Si <= 0.50
|   |   |   |   |   |   |   |--- r_gradiente_conocimiento_carrera <= -1.56
|   |   |   |   |   |   |   |   |--- alu_tec_espacio_fisico_2. Casi siempre <= 0.50
|   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |--- alu_tec_espacio_fisico_2. Casi siempre >  0.50
|   |   |   |   |   |   |   |   |   |--- r_cantidad_idiomas_buenos <= -0.03
|   |   |   |   |   |   |   |   |   |   |--- mot_inst_conozco_inst_S <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: 1.0
|   |   |   |   |   |   |   |   |   |   |--- mot_inst_conozco_inst_

11.b) Feature importance: recurro a distintas procedimientos que me sirven para ver qué variables independientes resultaron significativos para los modelos.

Genero una lista con todas esas variables que de una u otra forma y para uno u otro modelo resultaron significativas.

Me quedo con el comienzo de los nombres de esas variables para, luego, limitar mi data frame a las variables que comienzan de esa forma. Esto último pensando en que como consecuencia de la transformación a dummies una variable como trabajo se transformó en trabajo_si, trabajo_no_pero_busco, trabajo_no_y_no_busco. Luego, si con el feature importance detecté como significativa la variable trabajo_si, mi objetivo es conservar todas las variables que se hermanan a trabajo_si por partir de la misma variable. Así, al decir que quiero un data frame que conserve toda variable que comience con 'trab', voy a conservar las tres variables.

<span style="color:blue">¿Esto es adecuado o podría haber conservado sólo trabajo_si y no preocuparme por las demás?

11.b.1) Genero un primer listado de variables relevantes

In [92]:
# miro la importancia de las distintas variables independientes a la hora de precedir la variable dependiente según los distintos modelos utilizados

dt_importance = pd.Series(dt_model.feature_importances_, 
                              index=variables_independientes_2).sort_values(ascending=False) 

vars_dt_importance = dt_importance.nlargest(5).index 

In [93]:
rf_importance = pd.Series(rf_model.feature_importances_, 
                              index=variables_independientes_2).sort_values(ascending=False) 

vars_rf_importance = rf_importance.nlargest(5).index 

In [94]:
xgb_importance = pd.Series(xgb_model.feature_importances_, 
                              index=variables_independientes_2).sort_values(ascending=False) 

vars_xgb_importance = xgb_importance.nlargest(5).index 

In [95]:
# acá vemos el feature importance en nuestra regresión logística en función de los coeficientes del modelo 
# porque el procedimiento anterior no nos sirve para este modelo

coefficients = pd.Series(lr_model.coef_[0], index=variables_independientes_2)

lr_importance = coefficients.abs().sort_values(ascending=False)

vars_lr_importance = lr_importance.nlargest(5).index 

In [96]:
# acá vemos el feature importance en nuestro knn en función de ¿las distancias?

from sklearn.inspection import permutation_importance

knn_permutation = permutation_importance(knn_model, X_test, y_test, n_repeats=3, random_state=42)

knn_importance = pd.Series(knn_permutation.importances_mean, index=variables_independientes_2).sort_values(ascending=False)

vars_knn_importance = knn_importance.nlargest(5).index

In [97]:
# hago una primera lista del feature importance de los 5 modelos

all_vars = pd.Series(list(vars_dt_importance) + list(vars_rf_importance) + 
                     list(vars_xgb_importance) + list(vars_lr_importance) + 
                     list(vars_knn_importance))

primer_conjunto_variables = all_vars.unique()

print(primer_conjunto_variables)

['r_edad' 'r_gradiente_conocimiento_carrera' 'r_cantidad_idiomas_buenos'
 'cuantos_medios_transportes_insume' 'mot_inst_prestigio_S'
 'tipo_vivienda_Casa / Departamento propio'
 'r_ult_est_cur_padremadre_Colegio Secundario incompleto'
 'alu_cos_est_beca_Sí' 'madre_vive_No'
 'alu_tec_espacio_fisico_2. Casi siempre'
 'r_mot_propuesta_principal_mot_prop_interes_ejercer_docencia'
 'r_mot_propuesta_principal_mot_prop_remuneracion_docente'
 'estado_civil_Separado' 'tipo_vivienda_Residencia estudiantil'
 'r_ult_est_cur_padremadre_Escuela primaria incompleta'
 'vive_actual_con_Otros' 'alu_estudio_previo4_Si'
 'r_mot_propuesta_principal_mot_prop_no_sabia_que_estudiar'
 'cuanto_tiempo_insumira_viajes_2. Entre 30 minutos y 1 hora']


11.b.2) Genero un segundo listado de variables relevantes

In [98]:
# busco relevancia de variables independientes de una forma distinta y genero una segunda lista con este feature importance

from sklearn.feature_selection import SelectKBest, f_classif

selector = SelectKBest(score_func=f_classif, k=5)

X_new = selector.fit_transform(X_train, y_train)

selected_features = selector.get_support(indices=True)

segundo_conjunto_variables = X_train.columns[selected_features]

print(segundo_conjunto_variables)

Index(['mot_inst_prestigio_S', 'mot_inst_ubicacion_S',
       'r_mot_propuesta_principal_mot_prop_remuneracion_docente',
       'r_ult_est_cur_padremadre_Escuela primaria incompleta',
       'r_ult_est_cur_padremadre_Est. de Post grado'],
      dtype='object')


11.b.3) Uno los dos listados de variables relevantes y me quedo con valores únicos

In [99]:
conjunto_completo = pd.Series(list(primer_conjunto_variables) + list(segundo_conjunto_variables))

conjunto_completo_unique = conjunto_completo.unique()

conjunto_completo_unique

array(['r_edad', 'r_gradiente_conocimiento_carrera',
       'r_cantidad_idiomas_buenos', 'cuantos_medios_transportes_insume',
       'mot_inst_prestigio_S', 'tipo_vivienda_Casa / Departamento propio',
       'r_ult_est_cur_padremadre_Colegio Secundario incompleto',
       'alu_cos_est_beca_Sí', 'madre_vive_No',
       'alu_tec_espacio_fisico_2. Casi siempre',
       'r_mot_propuesta_principal_mot_prop_interes_ejercer_docencia',
       'r_mot_propuesta_principal_mot_prop_remuneracion_docente',
       'estado_civil_Separado', 'tipo_vivienda_Residencia estudiantil',
       'r_ult_est_cur_padremadre_Escuela primaria incompleta',
       'vive_actual_con_Otros', 'alu_estudio_previo4_Si',
       'r_mot_propuesta_principal_mot_prop_no_sabia_que_estudiar',
       'cuanto_tiempo_insumira_viajes_2. Entre 30 minutos y 1 hora',
       'mot_inst_ubicacion_S',
       'r_ult_est_cur_padremadre_Est. de Post grado'], dtype=object)

11.b.4) Me quedo sólo con los primeros 8 caracteres de los nombres de las variables relevantes seleccionadas

Sucede que muchas de estas variables son dummies. Algunas de ellas quedaron en el listado, por caso "r_mot_propuesta_principal_mot_prop_interes_ejercer_docencia". Mi idea es poder quedarme con todas las variables que nacieron de la misma variable original "r_mot_propuesta_principal". Entonces me quedo sólo con el comienzo del nombre de la variable para que luego me las traiga todas.

In [100]:
conjunto_completo_unique_comienzo = [var[:8] for var in conjunto_completo_unique]

conjunto_completo_unique_comienzo_unique = pd.Series(conjunto_completo_unique_comienzo).unique()

conjunto_completo_unique_comienzo_unique

array(['r_edad', 'r_gradie', 'r_cantid', 'cuantos_', 'mot_inst',
       'tipo_viv', 'r_ult_es', 'alu_cos_', 'madre_vi', 'alu_tec_',
       'r_mot_pr', 'estado_c', 'vive_act', 'alu_estu', 'cuanto_t'],
      dtype=object)

# 12) Hago feature selection: selecciono las variables independientes que le voy a pasar a los modelos definitivos

In [101]:
variables_seleccionadas_def = []

for col in data_1_imputado.columns:
    for seleccionar in conjunto_completo_unique_comienzo_unique:
         if col.startswith(seleccionar):
             variables_seleccionadas_def.append(col)

print(variables_seleccionadas_def)

['cuantos_medios_transportes_insume', 'r_edad', 'r_gradiente_conocimiento_carrera', 'r_cantidad_idiomas_buenos', 'mot_inst_publica_S', 'mot_inst_difusion_S', 'mot_inst_rec_amigos_S', 'mot_inst_oferta_unica_S', 'mot_inst_plan_estudio_S', 'mot_inst_prestigio_S', 'mot_inst_rec_estud_S', 'mot_inst_ubicacion_S', 'mot_inst_turno_S', 'mot_inst_conozco_inst_S', 'mot_inst_sist_ing_S', 'mot_inst_otros_S', 'tipo_vivienda_Casa / Departamento propio', 'tipo_vivienda_Otros', 'tipo_vivienda_Pensión', 'tipo_vivienda_Residencia estudiantil', 'vive_actual_con_Con familia de origen (padres, hermanos, abuelos)', 'vive_actual_con_Con su pareja/hijos', 'vive_actual_con_Otros', 'vive_actual_con_Solo', 'cuanto_tiempo_insumira_viajes_2. Entre 30 minutos y 1 hora', 'cuanto_tiempo_insumira_viajes_3. Entre 1 hora y 1 hora y media', 'cuanto_tiempo_insumira_viajes_4. Más de 1 hora y media', 'alu_cos_est_ap_fam_S', 'alu_cos_est_plsoc_S', 'alu_cos_est_trab_S', 'alu_cos_est_beca_Sí', 'alu_cos_est_ahorro_Sí', 'alu_cos_

In [102]:
data_1_imputado_def = data_1_imputado[variables_seleccionadas_def + ['r_desercion_temprana']]

print("Corroboro que hayan disminuido las variables")
print(data_1_imputado.shape)
print(data_1_imputado_def.shape)
print("Veo qué variables quedaron, atentos a que no se repitan ni nada raro")
print(data_1_imputado_def.columns)

Corroboro que hayan disminuido las variables
(6010, 106)
(6010, 71)
Veo qué variables quedaron, atentos a que no se repitan ni nada raro
Index(['cuantos_medios_transportes_insume', 'r_edad',
       'r_gradiente_conocimiento_carrera', 'r_cantidad_idiomas_buenos',
       'mot_inst_publica_S', 'mot_inst_difusion_S', 'mot_inst_rec_amigos_S',
       'mot_inst_oferta_unica_S', 'mot_inst_plan_estudio_S',
       'mot_inst_prestigio_S', 'mot_inst_rec_estud_S', 'mot_inst_ubicacion_S',
       'mot_inst_turno_S', 'mot_inst_conozco_inst_S', 'mot_inst_sist_ing_S',
       'mot_inst_otros_S', 'tipo_vivienda_Casa / Departamento propio',
       'tipo_vivienda_Otros', 'tipo_vivienda_Pensión',
       'tipo_vivienda_Residencia estudiantil',
       'vive_actual_con_Con familia de origen (padres, hermanos, abuelos)',
       'vive_actual_con_Con su pareja/hijos', 'vive_actual_con_Otros',
       'vive_actual_con_Solo',
       'cuanto_tiempo_insumira_viajes_2. Entre 30 minutos y 1 hora',
       'cuanto_tiempo_i

# 13) Salgo de mi data frame y y vuelvo a crear los objetos y e X

In [103]:
y = data_1_imputado_def['r_desercion_temprana']
X = data_1_imputado_def.drop('r_desercion_temprana', axis=1)

# 14) Repito el split train-test con mis nuevos y e X

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=91, stratify=y)
print("Vemos que los train tienen más filas que los test, pero tienen la misma cantidad de columnas")
print(y_train.shape)
print(y_test.shape)
print(X_train.shape)
print(X_test.shape)

Vemos que los train tienen más filas que los test, pero tienen la misma cantidad de columnas
(4808,)
(1202,)
(4808, 70)
(1202, 70)


In [105]:
# confirmo que sigo sin valores nulos

nulos_X_train = X_train.isnull().sum()
nulos_y_train = y_train.isnull().sum()

print("Valores nulos en X_train:")
print(nulos_X_train.sort_values(ascending=False))

print("Valores nulos en y_train:")
print(nulos_y_train)

Valores nulos en X_train:
cuantos_medios_transportes_insume                                       0
alu_tec_espacio_fisico_4. Nunca                                         0
alu_tec_calidad_conec_Muy buena                                         0
alu_tec_calidad_conec_Mala                                              0
alu_tec_tipo_conec_No cuento con conexión a internet en mi domicilio    0
                                                                       ..
alu_cos_est_trab_S                                                      0
alu_cos_est_beca_Sí                                                     0
alu_cos_est_ahorro_Sí                                                   0
alu_cos_est_otra_S                                                      0
r_ult_est_cur_padremadre_No hizo estudios                               0
Length: 70, dtype: int64
Valores nulos en y_train:
0


# 15) Optimización hiperparámetros con Grid Search

In [106]:
def optimize_model(model, param_grid, X_train, y_train):

    print(f"Optimizando modelo: {model.__class__.__name__}...")
    
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=5,
        scoring='accuracy',
        n_jobs=1
    )
    
    grid_search.fit(X_train, y_train)
    
    best_params = grid_search.best_params_
    
    print(f"Mejores parámetros: {best_params}")
    
    return best_params

Almaceno los mejores parametros que encuentro con Grid Search para cada modelo para, luego, poder usarlos cuando corra los modelos

In [107]:
dt_model = DecisionTreeClassifier(random_state=42)

dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

dt_best_params = optimize_model(
    model=dt_model,
    param_grid=dt_param_grid,
    X_train=X_train,
    y_train=y_train
)

Optimizando modelo: DecisionTreeClassifier...
Mejores parámetros: {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}


In [108]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)

rf_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'criterion': ['gini']
}

rf_best_params = optimize_model(
    model=rf_model,
    param_grid=rf_param_grid,
    X_train=X_train,
    y_train=y_train
)

Optimizando modelo: RandomForestClassifier...
Mejores parámetros: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [109]:
xgb_model = XGBClassifier(eval_metric='logloss', random_state=42)

xgb_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'gamma': [0, 1]
}

xgb_best_params = optimize_model(
    model=xgb_model,
    param_grid=xgb_param_grid,
    X_train=X_train,
    y_train=y_train
)

Optimizando modelo: XGBClassifier...
Mejores parámetros: {'colsample_bytree': 0.8, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}


In [110]:
lr_model = LogisticRegression(random_state=42)

lr_param_grid = {
    'penalty': ['l1', 'l2', 'none'],  
    'C': [0.1, 1],
    'solver': ['lbfgs', 'liblinear'],
    'max_iter': [100]
}
lr_best_params = optimize_model(
    model=lr_model,
    param_grid=lr_param_grid,
    X_train=X_train,
    y_train=y_train
)

Optimizando modelo: LogisticRegression...
Mejores parámetros: {'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

In [111]:
knn_model = KNeighborsClassifier()

knn_param_grid = {
    'n_neighbors': [3, 5, 10, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_best_params = optimize_model(
    model=knn_model,
    param_grid=knn_param_grid,
    X_train=X_train,
    y_train=y_train
)

Optimizando modelo: KNeighborsClassifier...
Mejores parámetros: {'metric': 'manhattan', 'n_neighbors': 15, 'weights': 'uniform'}


# 16) Hago el modelado definitivo

16.a) Defino los modelos ajutando hiperparametros y ejecuto la función que creé en el pasado

Árbol de decisión

In [112]:
results_3 = {}

In [113]:
dt_model = DecisionTreeClassifier(**dt_best_params,random_state=42)

dt_model, dt_pred, dt_accuracy, dt_cross_val_mean = evaluate_model(dt_model, X_train, X_test, y_train, y_test)

results_3['Arbol de decision 3'] = (dt_accuracy, dt_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 39 440]
 [ 67 656]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5782029950083195
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5066392467587763
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5782029950083195
El F1 muestra la media armónica de la precisión y el recall:
0.48697903469783577
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.613574                  0.0
2807     0.0     0.0      0.482385                  0.0
4336     1.0     0.0      0.482385                  1.0
1681     1.0     1.0      0.613574                  0.0
1881     1.0     0.0      0.482385                  1

Random Forest

In [114]:
rf_model = RandomForestClassifier(**rf_best_params, class_weight='balanced', random_state=42)   
                                                                          
rf_model, rf_pred, rf_accuracy, rf_cross_val_mean = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

results_3['Random Forest 3'] = (rf_accuracy, rf_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 68 411]
 [109 614]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.56738768718802
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5134086797141414
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.56738768718802
El F1 muestra la media armónica de la precisión y el recall:
0.5051786909447585
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.505703                  0.0
2807     0.0     0.0      0.406657                  0.0
4336     1.0     1.0      0.560000                  0.0
1681     1.0     1.0      0.720000                  0.0
1881     1.0     1.0      0.653989                  0.0
..

XG Boost

In [115]:
xgb_model = XGBClassifier(**xgb_best_params, eval_metric='mlogloss', random_state=42) 

xgb_model, xgb_pred, xgb_accuracy, xgb_cross_val_mean = evaluate_model(xgb_model, X_train, X_test, y_train, y_test)

results_3['XG Boost 3'] = (xgb_accuracy, xgb_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[  8 471]
 [ 30 693]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5831946755407654
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.44200331243911617
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5831946755407654
El F1 muestra la media armónica de la precisión y el recall:
0.45413220449611613
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0       1      0.560251                  0.0
2807     0.0       1      0.501773                  1.0
4336     1.0       1      0.545021                  0.0
1681     1.0       1      0.633918                  0.0
1881     1.0       1      0.503825                  

Regresión Logística

In [116]:
lr_model = LogisticRegression(**lr_best_params, random_state=42)

lr_model, lr_pred, lr_accuracy, lr_cross_val_mean = evaluate_model(lr_model, X_train, X_test, y_train, y_test)

results_3['Regresión Logística 3'] = (lr_accuracy, lr_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[  0 479]
 [  1 722]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.6006655574043261
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.36159966528170506
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.6006655574043261
El F1 muestra la media armónica de la precisión y el recall:
0.45143575676021597
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.601144                  0.0
2807     0.0     1.0      0.589780                  1.0
4336     1.0     1.0      0.640700                  0.0
1681     1.0     1.0      0.616943                  0.0
1881     1.0     1.0      0.561866                  

KNN

In [117]:
knn_model = KNeighborsClassifier(**knn_best_params)

knn_model, knn_pred, knn_accuracy, knn_cross_val_mean = evaluate_model(knn_model, X_train, X_test, y_train, y_test)

results_3['KNN 3'] = (knn_accuracy, knn_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[106 373]
 [146 577]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5682196339434277
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.5329546550791843
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5682196339434277
El F1 muestra la media armónica de la precisión y el recall:
0.5304714261908867
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     0.0      0.466667                  1.0
2807     0.0     1.0      0.533333                  1.0
4336     1.0     1.0      0.600000                  0.0
1681     1.0     1.0      0.733333                  0.0
1881     1.0     1.0      0.666667                  0.

# 17) Ensamblo los modelos definitivos para ver si obtengo mejores métricas de evaluación que con los modelos por separado

In [118]:
voting_soft_clf = VotingClassifier(estimators=[ 
    ('dt', dt_model),                                
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lr', lr_model),
    ('knn', knn_model)
], voting='soft') 

voting_soft_clf, voting_pred, voting_accuracy, voting_cross_val_mean = evaluate_model(voting_soft_clf, X_train, X_test, y_train, y_test)

results_3['Ensamble voting 3'] = (voting_accuracy, voting_cross_val_mean)

La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[ 10 469]
 [ 16 707]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.5965058236272879
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.514884761660968
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.5965058236272879
El F1 muestra la media armónica de la precisión y el recall:
0.46365877568428865
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.549468                  0.0
2807     0.0     1.0      0.502786                  1.0
4336     1.0     1.0      0.565621                  0.0
1681     1.0     1.0      0.663554                  0.0
1881     1.0     1.0      0.573746                  0.

In [119]:
stacking_clf = StackingClassifier(estimators=[ 
    ('dt', dt_model),                                
    ('rf', rf_model),
    ('xgb', xgb_model),
    ('lr', lr_model),
    ('knn', knn_model)],    
    final_estimator=LogisticRegression(),    
    cv=5)

stacking_clf, stacking_pred, stacking_accuracy, stacking_cross_val_mean = evaluate_model(stacking_clf, X_train, X_test, y_train, y_test)

results_3['Ensamble stacking 3'] = (stacking_accuracy, stacking_cross_val_mean)

c:\Users\facundo.troyer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


La matriz de confusión muestra la cantidad de verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos:
[[  0 479]
 [  0 723]]
El accuracy (o exactitud) muestra la proporción de predicciones correctas sobre el total de predicciones:
0.6014975041597338
El precision muestra l proporción de verdaderos positivos sobre el total de predicciones positivas:
0.36179924751038894
El recall (o sensibilidad) muestra la proporción de verdaderos positivos sobre el total de verdaderos positivos y falsos negativos:
0.6014975041597338
El F1 muestra la media armónica de la precisión y el recall:
0.45182617715063633
Visualización tabular de los resultados:
      y_test  y_pred  y_pred_proba  residuo_al_cuadrado
3859     1.0     1.0      0.587194                  0.0
2807     0.0     1.0      0.553788                  1.0
4336     1.0     1.0      0.574038                  0.0
1681     1.0     1.0      0.616314                  0.0
1881     1.0     1.0      0.558891                  

# 18) Genero una última vista resumen de algunas de las métricas de evaluación de los modelos

In [120]:
vista_resultados_3 = pd.DataFrame.from_dict(results_3, orient='index', columns=['Accuracy', 'Cross Val Mean'])

vista_resultados_3.reset_index(inplace=True)

vista_resultados_3.columns = ['Model', 'Accuracy', 'Cross Val Mean']

print(vista_resultados_3)

                   Model  Accuracy  Cross Val Mean
0    Arbol de decision 3  0.578203        0.594010
1        Random Forest 3  0.567388        0.578869
2             XG Boost 3  0.583195        0.592845
3  Regresión Logística 3  0.600666        0.601165
4                  KNN 3  0.568220        0.566389
5      Ensamble voting 3  0.596506        0.598003
6    Ensamble stacking 3  0.601498        0.601165


18.b) Junto y visualizo el accuracy y el cross val mean del primer, segundo y tercer modelado

In [121]:
vista_resultados_combined = pd.concat([vista_resultados, vista_resultados_2, vista_resultados_3], ignore_index=True)

print(vista_resultados_combined)

                    Model  Accuracy  Cross Val Mean
0       Arbol de decision  0.519040        0.518638
1           Random Forest  0.586093        0.586221
2                XG Boost  0.556291        0.554749
3     Regresion logistica  0.552980        0.555742
4                     KNN  0.515728        0.534709
5         Ensamble voting  0.556291        0.558228
6     Arbol de decision 2  0.507488        0.525790
7         Random Forest 2  0.559900        0.586023
8              XG Boost 2  0.543261        0.557903
9   Regresion logistica 2  0.577371        0.586522
10                  KNN 2  0.540765        0.539101
11      Ensamble voting 2  0.533278        0.558236
12    Arbol de decision 3  0.578203        0.594010
13        Random Forest 3  0.567388        0.578869
14             XG Boost 3  0.583195        0.592845
15  Regresión Logística 3  0.600666        0.601165
16                  KNN 3  0.568220        0.566389
17      Ensamble voting 3  0.596506        0.598003
18    Ensamb

<span style="color:red">Conclusiones:

<span style="color:red">Como se puede ver en los distintos accuray y promedio de validaciones se alcanzó una mejora a lo largo del trabajo, pero no en la magnitud que hubiesemos deseado.

<span style="color:red">A ello se suma que los últimos xg boost y regresión logística empeoraron mucho en precision y F1.

<span style="color:red">Como plasmamos en las preguntas que dejamos a lo largo del trabajo en color azul, nos cuestimos si acaso algunas de las decisiones que tomamos no deberían haber sido distintas.

<span style="color:red">No obstante, se alcanza el objetivo de mejorar las métricas y creemos que el último random forest o alguno de los ensambles podría ser de utilidad para predecir la deserción temprana en función de los atributos de los ingresantes.</span>